In [1]:
#!pip install opencv-python
#!pip install numpy==1.23
import torch
import numpy as np
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from pyqtree import Index
from scipy import stats
from rtree import index
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from template_mesh import Template_Mesh
from geometry import Edge_Set
from scipy.spatial import distance
from PIL import Image
from skimage.metrics import structural_similarity as ssim

In [2]:
point_number = 5233
face_number = 10216

face_start_line = 2
face_end_line = 10218
point_start_line = 10219
point_end_line = 15452
contour_start_line = 15455
contour_end_line = 15655
boundary_start_line = 15657
boundary_end_line = 15707

In [3]:
# x,y-coordinates for each point
def read_points(filename):
    lines = open(filename, 'r').readlines()
    points = np.zeros([point_number, 2], dtype=float)
    for i, line in enumerate(lines[point_start_line : point_end_line]):
        line = line.strip().split('\t')
        points[i,0] = float(line[0])
        points[i,1] = float(line[1])
    return points

def read_faces(filename):
    lines = open(filename, 'r').readlines()
    faces = np.zeros([face_number, 3], dtype=int)
    for i, line in enumerate(lines[face_start_line : face_end_line]):
        line = line.strip().split('\t')
        faces[i,0] = int(line[1])
        faces[i,1] = int(line[2])
        faces[i,2] = int(line[3])
    return faces

# on airfoil surface
def read_contour_id_list(filename):
    lines = open(filename, 'r').readlines()
    contour_id_list = []
    for line in lines[contour_start_line : contour_end_line]:
        line = line.strip().split('\t')
        contour_id_list.append([int(line[1]), int(line[2])])
    return np.array(contour_id_list, dtype=int)

def read_boundary_id_list(filename):
    lines = open(filename, 'r').readlines()
    boundary_id_list = []
    for line in lines[boundary_start_line : boundary_end_line]:
        line = line.strip().split('\t')
        boundary_id_list.append([int(line[1]), int(line[2])])
    return np.array(boundary_id_list, dtype=int)

def read_pressure_coefficient(filename):
    df = pd.read_csv(filename)
    pressure_coefficient_list = np.array(df['Pressure_Coefficient'])
    return pressure_coefficient_list    

In [4]:
FOLDER1 = 'mesh/'
FOLDER2 = 'restart_csv/'

def read_all_data(FILENAME1, FILENAME2):
    
    FILE1 = FOLDER1 + FILENAME1
    FILE2 = FOLDER2 + FILENAME2
    
    points = read_points(FILE1)
    triangles = read_faces(FILE1)
    contour = read_contour_id_list(FILE1) # 200
    boundary = read_boundary_id_list(FILE1) # 50
    pressure_coefs = read_pressure_coefficient(FILE2)
    
    return points, triangles, contour, boundary, pressure_coefs

In [5]:
points_0013, triangles_0013, contour_0013, boundary_0013, pressure_coefs_0013 = read_all_data('0013.su2', '0013.csv')
points_0719, triangles_0719, contour_0719, boundary_0719, pressure_coefs_0719 = read_all_data('0719.su2', '0719.csv')

In [270]:
# READING IN MANY FILES
def get_identifier(filename):
    """Extract identifier fron the filename."""
    return filename.split('.')[0]

def read_in_many_files(FOLDER1, FOLDER2):
    files_in_folder_1 = os.listdir(FOLDER1)
    data = {}
    for file1 in files_in_folder_1:
        identifier = get_identifier(file1)
        file2 = str(identifier) + '.csv'
        if file2 in os.listdir(FOLDER2):
            points, triangles, contour, boundary, pressure_coefs = read_all_data(file1, file2)
        # store in dictionary
        data[identifier] = {  
                'points': points,
                'triangles': triangles,
                'contour': contour,
                'boundary': boundary,
                'pressure_coefs': pressure_coefs}

    return data

In [280]:
# SIZE
FOLDER1 = 'mesh/size_a/'  # TO CHANGE
FOLDER2 = 'restart_csv/size_a/'  # TO CHANGE
data_size_a = read_in_many_files(FOLDER1, FOLDER2)

FOLDER1 = 'mesh/size_b/'  # TO CHANGE
FOLDER2 = 'restart_csv/size_b/'  # TO CHANGE
data_size_b = read_in_many_files(FOLDER1, FOLDER2)

# Help Functions

In [6]:
def precompute_triangle_values(triangles, points):  # triangle indices
    """
    Calculate and store some values for all triangles.
    
    Returns: Array of intermediate values for each triangle
    """
    num_triangles = len(triangles)
    triangle_values = np.zeros((num_triangles, 8))
    
    for t, triangle in enumerate(triangles):
        A = points[triangle[0]]  # 坐标
        B = points[triangle[1]]
        C = points[triangle[2]]
        
        
        v0 = C - A # vector
        v1 = B - A

        dot00 = np.dot(v0,v0)
        dot01 = np.dot(v0,v1)
        dot11 = np.dot(v1,v1)
        
        invDenom = 1 / (dot00 * dot11 - dot01 * dot01)
        
        triangle_values[t] = [v0[0], v0[1], v1[0], v1[1], invDenom, dot00, dot01, dot11]
    
    return triangle_values

In [7]:
# used in find_triangles_for_points
def create_spatial_index(triangles, points):
    # Find bounds of all triangles
    bbox = (min(points[:,0]), min(points[:,1]), max(points[:,0]), max(points[:,1]))
    
    # Create a spatial index using Quadtree
    spatial_index = Index(bbox)
    
    # Insert each triangle into the index
    for i, triangle in enumerate(triangles):
        t_points = points[triangle]
        t_bbox = (min(t_points[:,0]), min(t_points[:,1]), max(t_points[:,0]), max(t_points[:,1]))
        spatial_index.insert(i, t_bbox)
        
    return spatial_index                 

In [8]:
# used in find_triangles_for_points
def is_point_inside_triangle(point, A, B, C): # A, B, C are coordinates
    
    # Compute vectors    
    v0 = np.array(C) - np.array(A)
    v1 = np.array(B) - np.array(A)
    v2 = np.array(point) - np.array(A)

    # Compute dot products
    dot00 = np.dot(v0, v0)
    dot01 = np.dot(v0, v1)
    dot02 = np.dot(v0, v2)
    dot11 = np.dot(v1, v1)
    dot12 = np.dot(v1, v2)

    # Compute barycentric coordinates
    invDenom = 1.0 / (dot00 * dot11 - dot01 * dot01)
    u = (dot11 * dot02 - dot01 * dot12) * invDenom
    v = (dot00 * dot12 - dot01 * dot02) * invDenom

    # Check if point is in triangle
    return (u >= 0) and (v >= 0) and (u + v <= 1)

In [9]:
def find_triangles_for_points(new_points, triangles, points):
    """
    Find the containing triangle for each point.
    
    Returns:
    - Dictionary mapping each new point to its containing triangle's indices.
    """
    spatial_index = create_spatial_index(triangles, points)
    point_to_triangle = {}
    
    for point in new_points:
        possible_triangles = list(spatial_index.intersect((point[0], point[1], point[0], point[1])))
        
        for tri_idx in possible_triangles:
            if is_point_inside_triangle(point, points[triangles[tri_idx][0]], points[triangles[tri_idx][1]], points[triangles[tri_idx][2]]):
                point_to_triangle[tuple(point)] = triangles[tri_idx]  # TUPLE
                break
    
    return point_to_triangle

In [10]:
def barycentric_coords(p, v1, v2, v3):
    """
    Compute the barycentric coordinates of point p with respect to triangle v1v2v3.
    
    Args:
    - A, B, C are coordinates of the vertices of the tirangle.
    
    Returns:
    - Array of barycentric coordinates (w1, w2, w3)
    """
    denominator = (v2[1] - v3[1])*(v1[0] - v3[0]) + (v3[0] - v2[0])*(v1[1] - v3[1])
    w1 = (v2[1] - v3[1])*(p[0] - v3[0]) + (v3[0] - v2[0])*(p[1] - v3[1])
    w2 = (v3[1] - v1[1])*(p[0] - v3[0]) + (v1[0] - v3[0])*(p[1] - v3[1])
    
    w1 = w1 / denominator
    w2 = w2 / denominator
    w3 = 1 - w1 - w2
    
    return np.array([w1, w2, w3])

In [11]:
def compute_barycentric_for_new_points(new_points, triangles, points):
    """
    Compute barycentric coordinates for a set of new points with respect to a set of triangles.
    
    Returns: 
    - an array containing barycentric data. Each row is [a, b, c, d, e, f], where
        - a, b, c are the indices of the triangle the new point belongs to
        - d, e, f are the barycentric coordinates of the new point w.r.t the triangle
    """
    point_to_triangle = find_triangles_for_points(new_points, triangles, points) # dictionary
    
    results = []
    
    for p in new_points:
        
        tri_indices = point_to_triangle[tuple(p)]
        A, B, C = points[tri_indices]
        b_coords = barycentric_coords(p, A, B, C)
        results.append(np.hstack((tri_indices, b_coords)))
    
    return np.array(results)

In [12]:
def interpolate_pressure_coef(new_points, triangles, points, pressure_coefs):
    """
    Interpolate pressure coefficients for the new points.
    """
    new_barycentric = compute_barycentric_for_new_points(new_points, triangles, points)  # 6D array
    
    interpolated_pressure_coefs = []
    
    for b in new_barycentric:
        tri_indices = b[:3].astype(int)
        bary_coords = b[3:]
        
        # get the pressure coefficients for the vertices
        vertex_pressure_coef = [pressure_coefs[idx] for idx in tri_indices]
        
        # new pressure coefs
        interpolated_value = sum(v * b for v,b in zip(vertex_pressure_coef, bary_coords))
        
        interpolated_pressure_coefs.append(interpolated_value)
        
    return np.array(interpolated_pressure_coefs)        

In [13]:
def computing_bounding_box(contour, points):
    """
    Compute the bounding box of the airfoil.
    
    Returns:
    - an array [min_x, max_x, min_y, max_y] representing the bounding box.
    """
    unique_indices = np.unique(contour)
    airfoil_points_coords = points[unique_indices]
    
    min_x, min_y = np.min(airfoil_points_coords, axis=0)
    max_x, max_y = np.max(airfoil_points_coords, axis=0)
    
    return np.array([min_x, max_x, min_y, max_y])  # 坐标值

In [14]:
def filter_points_within_bbox(grid_points, contour, points):
    """
    Return only grid points that lie within the bounding box, also their indices w.r.t grid_points.
    """
    min_x, max_x, min_y, max_y = computing_bounding_box(contour, points)
    
    filtered_indices = np.where((grid_points[:,0] >= min_x) &
                                (grid_points[:,0] <= max_x) &
                                (grid_points[:,1] >= min_y) &
                                (grid_points[:,1] <= max_y))

    return grid_points[filtered_indices], list(filtered_indices[0])

In [15]:
def is_point_inside_airfoil(p, contour, points):
    """
    Check if a point is inside the airfoil contour using ray casting method.
    """
    x, y = p
    intersections = 0
    for i in range(len(contour)):
        a = points[contour[i][0]] # 坐标
        b = points[contour[i][1]]
        
        # Check if ray intersects with edge of the contour
        if ((a[1] > y) != (b[1] > y)) and (x < (b[0] - a[0]) * (y - a[1]) / (b[1] - a[1]) + a[0]):
            intersections += 1
            
    # Return True if intersections count is odd
    return intersections % 2 == 1

In [16]:
def filter_points_outside_airfoil(contour, grid_points, points): 
    """
    Returns:
    - Array of coordinates of grid points that are inside the airfoil
    """
    # all points and their indices within the bounding box of the airfoil (w.r.t grid_points)
    bbox_points, bbox_indices = filter_points_within_bbox(grid_points, contour, points)
    
    # get the points that are inside the airfoil
    inside_airfoil_points = [p for p in bbox_points if is_point_inside_airfoil(p, contour, points)]
    inside_airfoil_tuples = {tuple(p) for p in inside_airfoil_points}
    inside_airfoil_indices = [bbox_indices[i] for i,p in enumerate(bbox_points) if tuple(p) in inside_airfoil_tuples]
    
    # get the points indices that are outside airfoil but inside bbox
    outside_airfoil_inside_bbox_indices = list(set(bbox_indices) - set(inside_airfoil_indices))
    
    # get the indices outside bbox
    outside_bbox_indices = list(set(range(grid_points.shape[0])) - set(bbox_indices))
    
    # combine the above two groups
    outside_airfoil_indices = outside_bbox_indices + outside_airfoil_inside_bbox_indices

    return grid_points[outside_airfoil_indices]

In [17]:
def filter_points_inside_airfoil(contour, grid_points, points):
    
    points_outside_airfoil = filter_points_outside_airfoil(contour, grid_points, points)
    set_of_all_points = set(map(tuple, grid_points))
    set_of_points_outside = set(map(tuple, points_outside_airfoil))
    points_inside_airfoil = list(set_of_all_points - set_of_points_outside)
    
    mask = np.zeros(grid_points.shape[0], dtype=bool)
    
    for i,p in enumerate(grid_points):
        if tuple(p) in points_inside_airfoil:
            mask[i] = True
    
    return mask
    #return np.array(points_inside_airfoil)

In [18]:
def get_points_inside_boundary(grid_points, points, boundary):
    
    boundary_coords = np.array([points[idx] for segment in boundary for idx in segment])
    path = Path(boundary_coords)
    mask = path.contains_points(grid_points)
    
    return grid_points[mask]

In [19]:
def filter_points_inside_boundary_outside_airfoil(grid_points, points, contour, boundary):
    
    points_inside_boundary = get_points_inside_boundary(grid_points, points, boundary)
    return filter_points_outside_airfoil(contour, points_inside_boundary, points)

In [20]:
def compute_pressure_coef_on_grid(points, triangles, pressure_coefs, contour, boundary, grid_points):
    """
    Compute pressure coefficient values for grid points that are outside of the airfoil and inside the boundary.
    """
    final_points = filter_points_inside_boundary_outside_airfoil(grid_points, points, contour, boundary)
    
    interpolated_pressure_coefs = interpolate_pressure_coef(final_points, triangles, points, pressure_coefs)
    
    return final_points, interpolated_pressure_coefs

In [21]:
# Color RGB values
WHITE = np.array([255,255,255])
BLACK = np.array([0,0,0])
BLUE = np.array([0,0,255]) 
GREEN = np.array([0,255,0])
RED = np.array([255,0,0])
PURPLE = np.array([127,0,255])
CYAN = np.array([0,255,255])
YELLOW = np.array([255,255,0])
PINK = np.array([255,0,255])
ORANGE = np.array([255,128,0])

PIYG_START = np.array([77, 146, 33])
PIYG_END = np.array([197, 27, 125])

PRGN_START = np.array([118, 42, 131])
PRGN_END = np.array([27, 120, 55])

SOFT_RED = np.array([255,51,51])
SOFT_BLUE = np.array([51,51,255])

In [317]:
def plot_pressure_on_grid(points, triangles, pressure_coefs, contour, boundary,  res, start_color, end_color, airfoil_color, x_min, x_max, y_min, y_max):
    
    xx, yy = np.meshgrid(           
        np.linspace(x_min, x_max, res),           
        np.linspace(y_min, y_max, res)    
    )
    grid_points = np.vstack([xx.ravel(), yy.ravel()]).T    
    
    final_points, interpolated_pressure_coefs = compute_pressure_coef_on_grid(points, triangles, pressure_coefs, contour, boundary, grid_points)
    
    pressure_grid = np.full(xx.shape, 0).astype(float)
    
    # create a look up map for the grid points
    grid_point_to_idx = {tuple(point): idx for idx,point in enumerate(grid_points)}
    
    # inside airfoil mask: check which points are inside the airfoil  
    contour_coords = points[np.unique(contour)] 
    airfoil_path = Path(contour_coords)
    inside_airfoil_mask = airfoil_path.contains_points(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    # normalized to [0,1]
    interpolated_pressure_coefs = interpolated_pressure_coefs - interpolated_pressure_coefs.min()
    interpolated_pressure_coefs /= interpolated_pressure_coefs.max() + 1e-12                                                                     
    
    # fill pressure_grid
    for pt, pressure in zip(final_points, interpolated_pressure_coefs):
        idx = grid_point_to_idx[tuple(pt)]
        ix, iy = np.unravel_index(idx, xx.shape)
        pressure_grid[ix, iy] = pressure
    
    # create an empty RGB image
    colored_image = np.zeros((*pressure_grid.shape, 3), dtype=np.uint8)  # np.uint8 ensures that the values are integers ranging from 0 to 255, which is standard for RGB images.
    
    for i in range(pressure_grid.shape[0]):
        for j in range(pressure_grid.shape[1]):
            if inside_airfoil_mask[i,j]:
                colored_image[i,j] = airfoil_color    
            else:
                value = pressure_grid[i,j]
                color = (1 - value) * start_color + value * end_color  # 压力小 start color, 压力大 end color
                colored_image[i,j] = color.astype(np.uint8) 
    
    cv2.imwrite('1.png', cv2.cvtColor(colored_image, cv2.COLOR_RGB2BGR))
       
def plot_pressure_on_grid_2(points, triangles, pressure_coefs, contour, boundary,  res, start_color, end_color, airfoil_color, x_min, x_max, y_min, y_max, identifier, FOLDER):
    xx, yy = np.meshgrid(           
        np.linspace(x_min, x_max, res),           
        np.linspace(y_min, y_max, res)    
    )
    grid_points = np.vstack([xx.ravel(), yy.ravel()]).T    
    final_points, interpolated_pressure_coefs = compute_pressure_coef_on_grid(points, triangles, pressure_coefs, contour, boundary, grid_points)
    pressure_grid = np.full(xx.shape, 0).astype(float)
    grid_point_to_idx = {tuple(point): idx for idx,point in enumerate(grid_points)} 
    contour_coords = points[np.unique(contour)] 
    airfoil_path = Path(contour_coords)
    inside_airfoil_mask = airfoil_path.contains_points(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    interpolated_pressure_coefs = interpolated_pressure_coefs - interpolated_pressure_coefs.min()
    interpolated_pressure_coefs /= interpolated_pressure_coefs.max() + 1e-12                                                                     
    for pt, pressure in zip(final_points, interpolated_pressure_coefs):
        idx = grid_point_to_idx[tuple(pt)]
        ix, iy = np.unravel_index(idx, xx.shape)
        pressure_grid[ix, iy] = pressure
    colored_image = np.zeros((*pressure_grid.shape, 3), dtype=np.uint8)  
    for i in range(pressure_grid.shape[0]):
        for j in range(pressure_grid.shape[1]):
            if inside_airfoil_mask[i,j]:
                colored_image[i,j] = airfoil_color    
            else:
                value = pressure_grid[i,j]
                color = (1 - value) * start_color + value * end_color
                colored_image[i,j] = color.astype(np.uint8) 
    if not os.path.exists(FOLDER):
        os.makedirs(FOLDER)
    filename = os.path.join(FOLDER, f'{identifier}.png')
    cv2.imwrite(filename, cv2.cvtColor(colored_image, cv2.COLOR_RGB2BGR))

In [319]:
def plot_black_and_white(points, triangles, pressure_coefs, contour, boundary, res, inside_color, outside_color, x_min, x_max, y_min, y_max):
    
    xx, yy = np.meshgrid(           
        np.linspace(x_min, x_max, res),            
        np.linspace(y_min, y_max, res)    
    )
    
    pressure_grid = np.full(xx.shape, 0).astype(float)
    
    # inside airfoil mask: check which points are inside the airfoil  
    contour_coords = points[np.unique(contour)] 
    airfoil_path = Path(contour_coords)
    inside_airfoil_mask = airfoil_path.contains_points(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    # create an empty RGB image
    bw_image = np.zeros((*pressure_grid.shape, 3), dtype=np.uint8) 
    
    for i in range(pressure_grid.shape[0]):
        for j in range(pressure_grid.shape[1]):
            if inside_airfoil_mask[i,j]:
                bw_image[i,j] = inside_color      
            else:
                bw_image[i,j] = outside_color
    
    cv2.imwrite('bw.png', cv2.cvtColor(bw_image, cv2.COLOR_RGB2BGR))

def plot_black_and_white_2(points, triangles, pressure_coefs, contour, boundary, res, inside_color, outside_color, x_min, x_max, y_min, y_max, identifier, FOLDER):
    xx, yy = np.meshgrid(           
        np.linspace(x_min, x_max, res),            
        np.linspace(y_min, y_max, res)    
    )
    pressure_grid = np.full(xx.shape, 0).astype(float)
    contour_coords = points[np.unique(contour)] 
    airfoil_path = Path(contour_coords)
    inside_airfoil_mask = airfoil_path.contains_points(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    bw_image = np.zeros((*pressure_grid.shape, 3), dtype=np.uint8) 
    for i in range(pressure_grid.shape[0]):
        for j in range(pressure_grid.shape[1]):
            if inside_airfoil_mask[i,j]:
                bw_image[i,j] = inside_color      
            else:
                bw_image[i,j] = outside_color
    if not os.path.exists(FOLDER):
        os.makedirs(FOLDER)
    filename = os.path.join(FOLDER, f'{identifier}_bw.png')
    cv2.imwrite(filename, cv2.cvtColor(bw_image, cv2.COLOR_RGB2BGR))

In [25]:
def plot_contour_black(points, contour, res, x_min, x_max, y_min, y_max):
    
    xx, yy = np.meshgrid(           
        np.linspace(x_min, x_max, res),            
        np.linspace(y_min, y_max, res)    
    )
    
    contour_coords = points[np.unique(contour)] 
    
    bw_image = np.full((*xx.shape, 3), 255, dtype=np.uint8) 
    
    for i in range(xx.shape[0]):
        for j in range(xx.shape[1]):
            point = [xx[i, j], yy[i, j]]
            
            min_distance = np.min(distance.cdist([point], contour_coords))

            if min_distance < 0.01:
                bw_image[i, j] = [0, 0, 0] 

    cv2.imwrite('contour.png', bw_image)

# Comparing the predicted image and the actual image

## Comparing how much the airfoil regions overlap

In [32]:
def extract_last_image(img_name, start_row=112, start_col=112):
    """Extract the last image from a 2x2 grid image."""
    img = cv2.imread(img_name)
    last_image = img[start_row:, start_col:]
    
    return last_image

In [54]:
img = extract_last_image("grid_1.png")
cv2.imwrite("extracted_image.png", img)

True

In [33]:
def create_binay_mask(image, threshold=50):
    """Create a binary mask to isolate the airfoil shape."""
    
    # Convert the image to grayscale if it's not already.
    if len(image.shape) == 3:
        grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else: 
        grayscale = image
    
    # Threshold the image to create a binary mask
    _,binary_mask = cv2.threshold(grayscale, threshold, 255, cv2.THRESH_BINARY_INV)
    
    return binary_mask
    # returns a binary image where the airfoil is white (255) and everything else is black (0)

In [34]:
def create_binay_mask_2(image, threshold=50):
   
    image = cv2.imread(image)  # difference compared to create_binary_mask
    
    if len(image.shape) == 3:
        grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else: 
        grayscale = image
    
    _,binary_mask = cv2.threshold(grayscale, threshold, 255, cv2.THRESH_BINARY_INV)
    
    return binary_mask
    # returns a binary image where the airfoil is white (255) and everything else is black (0)

In [329]:
def compare_airfoil_overlap(predicted_img, actual_img):
    
    # read image in grayscale
    actual_img = cv2.imread(actual_img, cv2.IMREAD_GRAYSCALE)
    predicted_img = cv2.imread(predicted_img, cv2.IMREAD_GRAYSCALE)
    
    # create binary masks for both images
    predicted_mask = create_binay_mask(predicted_img)
    actual_mask = create_binay_mask(actual_img)
    
    # convert the masks to boolean arrays
    predicted_bool = (predicted_mask > 0)
    actual_bool = (actual_mask > 0)
    
    # compute overlap 
    overlap = np.logical_and(predicted_bool, actual_bool)
    # compute union
    union = np.logical_or(predicted_bool, actual_bool)
    
    # calculate iou
    iou = np.sum(overlap) / np.sum(union)
    
    # calculate the overlap percentage
    total_pixels = np.prod(actual_bool.shape)
    overlap_pixels = np.sum(overlap)
    overlap_percentage = (overlap_pixels / total_pixels) * 100
    
    actual_airfoil_percentage = actual_bool.sum() / total_pixels * 100    
    
    return iou

## Comparing the generated image with the actual image

### Visual inspection

In [200]:
def overlay_difference(image_path1, image_path2, output_path='difference.png'):
    image1 = cv2.imread(image_path1)
    image2 = cv2.imread(image_path2)
    
    difference = cv2.absdiff(image1, image2)
    
    cv2.imwrite(output_path, difference)
    # areas will be black if difference is 0

In [40]:
overlay_difference('extracted_image.png','actual_image.png')

### Using MSE

In [105]:
def compute_mse(image_path1, image_path2):
    image1 = cv2.imread(image_path1, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image_path2, cv2.IMREAD_GRAYSCALE)
    
    diff = cv2.subtract(image1, image2)
    squared_diff = np.square(diff)
    mse = squared_diff.mean()
    
    return mse

In [43]:
compute_mse('extracted_image.png','actual_image.png')

99.99832589285714

### Using SSIM

In [42]:
def compute_ssim(image1_path, image2_path):
    image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)
    
    score, _ = ssim(image1, image2, full=True)

    return score

In [46]:
compute_ssim('extracted_image.png','actual_image.png')

0.7756733528065535

A value close to 1 means the two images are very similar. A value approacing 0 means significant differences between the images. 

### Using R^2

In [400]:
def compute_r_squared(predicted_image_path, actual_image_path):
    actual_img = cv2.imread(actual_image_path, cv2.IMREAD_GRAYSCALE)
    predicted_img = cv2.imread(predicted_image_path, cv2.IMREAD_GRAYSCALE)
    
    residuals = actual_img - predicted_img
    ss_res = np.sum(residuals**2) # sum of squares of the residuals
    ss_tot = np.sum((actual_img - np.mean(actual_img))**2)  # total sum of squares
    r_squared = 1 - (ss_res / ss_tot)
    
    return r_squared

In [49]:
compute_r_squared('actual_image.png', 'extracted_image.png')

'0.9621'

### Functions Loop Version

In [301]:
def plot_presure_on_grid_LOOP(data, res, start_color, end_color, airfoil_color, x_min, x_max, y_min, y_max, FOLDER):
    
    for identifier, dataset in data.items():
        points = dataset['points']
        triangles = dataset['triangles']
        pressure_coefs = dataset['pressure_coefs']
        contour = dataset['contour']
        boundary = dataset['boundary']
        plot_pressure_on_grid_2(points, triangles, pressure_coefs, contour, boundary, res, start_color, end_color, airfoil_color, x_min, x_max, y_min, y_max, identifier, FOLDER)

def plot_black_and_white_LOOP(data, res, inside_color, outside_color, x_min, x_max, y_min, y_max, FOLDER):
    for identifier, dataset in data.items():
        points = dataset['points']
        triangles = dataset['triangles']
        pressure_coefs = dataset['pressure_coefs']
        contour = dataset['contour']
        boundary = dataset['boundary']
        plot_black_and_white_2(points, triangles, pressure_coefs, contour, boundary, res, inside_color, outside_color, x_min, x_max, y_min, y_max, identifier, FOLDER)
        
def extract_last_image_LOOP(folder_path, output_folder, start_row=112, start_col=112):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        last_image = img[start_row:, start_col:]
        output_path = os.path.join(output_folder, f'{filename}')
        cv2.imwrite(output_path, last_image)

def compare_airfoil_overlap_LOOP(predicted_folder, actual_folder):
    predicted_files = sorted([f for f in os.listdir(predicted_folder) if f.lower().endswith('.png')])  # IMPORTANT 这里是按顺序的
    actual_files = sorted([f for f in os.listdir(actual_folder) if f.lower().endswith('.png')])
    if len(predicted_files) != len(actual_files):
        print("The number doesn't match.")
    results = []
    for predicted_file, actual_file in zip(predicted_files, actual_files):
        predicted_path = os.path.join(predicted_folder, predicted_file)
        actual_path = os.path.join(actual_folder, actual_file)
        iou = compare_airfoil_overlap(predicted_path, actual_path)
        results.append(iou)
    return np.array(results)

def compute_mse_LOOP(predicted_folder, actual_folder):
    predicted_files = sorted([f for f in os.listdir(predicted_folder) if f.lower().endswith('.png')]) 
    actual_files = sorted([f for f in os.listdir(actual_folder) if f.lower().endswith('.png')])
    if len(predicted_files) != len(actual_files):
        print("The number doesn't match.")
    results = []
    for predicted_file, actual_file in zip(predicted_files, actual_files):
        predicted_path = os.path.join(predicted_folder, predicted_file)
        actual_path = os.path.join(actual_folder, actual_file)
        mse = compute_mse(predicted_path, actual_path)
        results.append(mse)
    return np.array(results)

def compute_ssim_LOOP(predicted_folder, actual_folder):
    predicted_files = sorted([f for f in os.listdir(predicted_folder) if f.lower().endswith('.png')]) 
    actual_files = sorted([f for f in os.listdir(actual_folder) if f.lower().endswith('.png')])
    if len(predicted_files) != len(actual_files):
        print("The number doesn't match.")
    results = []
    for predicted_file, actual_file in zip(predicted_files, actual_files):
        predicted_path = os.path.join(predicted_folder, predicted_file)
        actual_path = os.path.join(actual_folder, actual_file)
        ssim = compute_ssim(predicted_path, actual_path)
        results.append(ssim)
    return np.array(results)

def compute_r_squared_LOOP(predicted_folder, actual_folder):
    predicted_files = sorted([f for f in os.listdir(predicted_folder) if f.lower().endswith('.png')]) 
    actual_files = sorted([f for f in os.listdir(actual_folder) if f.lower().endswith('.png')])
    if len(predicted_files) != len(actual_files):
        print("The number doesn't match.")
    results = []
    for predicted_file, actual_file in zip(predicted_files, actual_files):
        predicted_path = os.path.join(predicted_folder, predicted_file)
        actual_path = os.path.join(actual_folder, actual_file)
        r_squared = compute_r_squared(predicted_path, actual_path)
        results.append(r_squared)
    return np.array(results)

# Airfoil Zoom-in Level

## Loop Version

Use`a` to predict `b`

In [294]:
FOLDER1 = 'mesh/size_a/'  # TO CHANGE
FOLDER2 = 'restart_csv/size_a/'  # TO CHANGE
data_size_a = read_in_many_files(FOLDER1, FOLDER2)

FOLDER1 = 'mesh/size_b/'  # TO CHANGE
FOLDER2 = 'restart_csv/size_b/'  # TO CHANGE
data_size_b = read_in_many_files(FOLDER1, FOLDER2)

**SMALL**

In [297]:
# 右上
plot_presure_on_grid_LOOP(data_size_a, 500, WHITE, BLACK, BLACK, -1, 2, -0.5, 0.5, 'size/a_small_500')
# 左上  
plot_black_and_white_LOOP(data_size_a, 500, BLACK, WHITE, -1, 2, -0.5, 0.5, 'size/a_small_500_bw')
# 左下
plot_black_and_white_LOOP(data_size_b, 500, BLACK, WHITE, -1, 2, -0.5, 0.5, 'size/b_small_500_bw')

Use model

In [316]:
input_folder = 'size/generated_grid/small'
output_folder = 'size/extracted/small'
extract_last_image_LOOP(input_folder, output_folder)

In [318]:
# 右下
plot_presure_on_grid_LOOP(data_size_b, 112, WHITE, BLACK, BLACK, -1, 2, -0.5, 0.5, 'size/LR_112/small')

**MEDIUM**

In [305]:
# 右上
plot_presure_on_grid_LOOP(data_size_a, 500, WHITE, BLACK, BLACK, -0.6, 1.6, -0.3, 0.3, 'size/a_medium_500')
# 左上  
plot_black_and_white_LOOP(data_size_a, 500, BLACK, WHITE, -0.6, 1.6, -0.3, 0.3, 'size/a_medium_500_bw')
# 左下
plot_black_and_white_LOOP(data_size_b, 500, BLACK, WHITE, -0.6, 1.6, -0.3, 0.3, 'size/b_medium_500_bw')

In [320]:
input_folder = 'size/generated_grid/middle'
output_folder = 'size/extracted/middle'
extract_last_image_LOOP(input_folder, output_folder)

In [324]:
# 右下
plot_presure_on_grid_LOOP(data_size_b, 112, WHITE, BLACK, BLACK, -0.6, 1.6, -0.3, 0.3, 'size/LR_112/middle')

**LARGE**

In [309]:
# 右上
plot_presure_on_grid_LOOP(data_size_a, 500, WHITE, BLACK, BLACK, -0.2, 1.2, -0.15, 0.15, 'size/a_large_500')
# 左上  
plot_black_and_white_LOOP(data_size_a, 500, BLACK, WHITE, -0.2, 1.2, -0.15, 0.15, 'size/a_large_500_bw')
# 左下
plot_black_and_white_LOOP(data_size_b, 500, BLACK, WHITE, -0.2, 1.2, -0.15, 0.15, 'size/b_large_500_bw')

In [322]:
input_folder = 'size/generated_grid/large'
output_folder = 'size/extracted/large'
extract_last_image_LOOP(input_folder, output_folder)

In [325]:
# 右下
plot_presure_on_grid_LOOP(data_size_b, 112, WHITE, BLACK, BLACK, -0.2, 1.2, -0.15, 0.15, 'size/LR_112/large')

### Compare metrics

**Airfoil Overlap**

In [419]:
predicted_folder = 'size/extracted/small'
actual_folder = 'size/LR_112/small'
iou_small = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

predicted_folder = 'size/extracted/medium'
actual_folder = 'size/LR_112/medium'
iou_medium = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

predicted_folder = 'size/extracted/large'
actual_folder = 'size/LR_112/large'
iou_large = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

diff_1_2 = iou_medium - iou_small
diff_2_3 = iou_large - iou_medium
increase_1_2 = np.sum(diff_1_2 > 0) / len(diff_1_2) * 100
increase_2_3 = np.sum(diff_2_3 > 0) / len(diff_2_3) * 100
print("Proportion of positive increase from small to medium: {:.2f}%".format(increase_1_2))
print("Proportion of positive increase from medium to large: {:.2f}%\n".format(increase_2_3))

t_statistic, p_value = stats.ttest_rel(iou_small, iou_medium)
print("t-statistic for small and medium: {:.4f}".format(t_statistic))
print("p-value for small and medium: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

t_statistic, p_value = stats.ttest_rel(iou_medium, iou_large)
print("t-statistic for medium and large: {:.4f}".format(t_statistic))
print("p-value for medium and large: {:.4f}".format(p_value))
print("SIGNIFICANT!")

Proportion of positive increase from small to medium: 85.00%
Proportion of positive increase from medium to large: 85.00%

t-statistic for small and medium: -4.4768
p-value for small and medium: 0.0003
SIGNIFICANT!

t-statistic for medium and large: -3.4258
p-value for medium and large: 0.0028
SIGNIFICANT!


**MSE**

In [424]:
predicted_folder = 'size/extracted/small'
actual_folder = 'size/LR_112/small'
mse_small = compute_mse_LOOP(predicted_folder, actual_folder)

predicted_folder = 'size/extracted/medium'
actual_folder = 'size/LR_112/medium'
mse_medium = compute_mse_LOOP(predicted_folder, actual_folder)

predicted_folder = 'size/extracted/large'
actual_folder = 'size/LR_112/large'
mse_large = compute_mse_LOOP(predicted_folder, actual_folder)

diff_1_2 = mse_medium - mse_small
diff_2_3 = mse_large - mse_medium
decrease_1_2 = np.sum(diff_1_2 < 0) / len(diff_1_2) * 100
decrease_2_3 = np.sum(diff_2_3 < 0) / len(diff_2_3) * 100
print("Proportion of positive decrease from small to medium: {:.2f}%".format(decrease_1_2))
print("Proportion of positive decrease from medium to large: {:.2f}%\n".format(decrease_2_3))

t_statistic, p_value = stats.ttest_rel(mse_small, mse_medium)
print("t-statistic for small and medium: {:.4f}".format(t_statistic))
print("p-value for small and medium: {:.4f}".format(p_value))
print("NOT SIGNIFICANT.\n")

t_statistic, p_value = stats.ttest_rel(mse_medium, mse_large)
print("t-statistic for medium and large: {:.4f}".format(t_statistic))
print("p-value for medium and large: {:.4f}".format(p_value))
print("SIGNIFICANT!")

Proportion of positive decrease from small to medium: 50.00%
Proportion of positive decrease from medium to large: 90.00%

t-statistic for small and medium: 1.0572
p-value for small and medium: 0.3037
NOT SIGNIFICANT.

t-statistic for medium and large: 6.9695
p-value for medium and large: 0.0000
SIGNIFICANT!


**R_squared**

In [426]:
predicted_folder = 'size/extracted/small'
actual_folder = 'size/LR_112/small'
r_small = compute_r_squared_LOOP(predicted_folder, actual_folder)

predicted_folder = 'size/extracted/medium'
actual_folder = 'size/LR_112/medium'
r_medium = compute_r_squared_LOOP(predicted_folder, actual_folder)

predicted_folder = 'size/extracted/large'
actual_folder = 'size/LR_112/large'
r_large = compute_r_squared_LOOP(predicted_folder, actual_folder)

diff_1_2 = r_medium - r_small
diff_2_3 = r_large - r_medium
increase_1_2 = np.sum(diff_1_2 > 0) / len(diff_1_2) * 100
increase_2_3 = np.sum(diff_2_3 > 0) / len(diff_2_3) * 100
print("Proportion of positive increase from small to medium: {:.2f}%".format(increase_1_2))
print("Proportion of positive increase from medium to large: {:.2f}%\n".format(increase_2_3))

t_statistic, p_value = stats.ttest_rel(r_small, r_medium)
print("t-statistic for small and medium: {:.4f}".format(t_statistic))
print("p-value for small and medium: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

t_statistic, p_value = stats.ttest_rel(r_medium, r_large)
print("t-statistic for medium and large: {:.4f}".format(t_statistic))
print("p-value for medium and large: {:.4f}".format(p_value))
print("SIGNIFICANT!")

Proportion of positive increase from small to medium: 100.00%
Proportion of positive increase from medium to large: 100.00%

t-statistic for small and medium: -4.4655
p-value for small and medium: 0.0003
SIGNIFICANT!

t-statistic for medium and large: -4.7435
p-value for medium and large: 0.0001
SIGNIFICANT!


## Single Version

Use 0013 to predict 0124. Change size only.

In [144]:
points_0124, triangles_0124, contour_0124, boundary_0124, pressure_coefs_0124 = read_all_data('0124.su2', '0124.csv')

In [303]:
## SMALL
# 右上
plot_pressure_on_grid(points_0013, triangles_0013, pressure_coefs_0013, contour_0013, boundary_0013, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-1, x_max=2, y_min=-0.5, y_max=0.5)
# 左上
plot_black_and_white(points_0013, triangles_0013, pressure_coefs_0013, contour_0013, boundary_0013, res=1000, inside_color=BLACK, outside_color=WHITE,  x_min=-1, x_max=2, y_min=-0.5, y_max=0.5)
# 右下
plot_black_and_white(points_0124, triangles_0124, pressure_coefs_0124, contour_0124, boundary_0124, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-1, x_max=2, y_min=-0.5, y_max=0.5)
# 右下
plot_black_and_white(points_0124, triangles_0124, pressure_coefs_0124, contour_0124, boundary_0124, res=112, inside_color=BLACK, outside_color=WHITE, x_min=-1, x_max=2, y_min=-0.5, y_max=0.5)

img = extract_last_image("size/grid_small_0124.png")
cv2.imwrite("size/extracted_small.png", img)

In [138]:
## MEDIUM
plot_black_and_white(points_0423, triangles_0423, pressure_coefs_0423, contour_0423, boundary_0423, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.3, y_max=0.3)
plot_black_and_white(points_0124, triangles_0124, pressure_coefs_0124, contour_0124, boundary_0124, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.3, y_max=0.3)
plot_pressure_on_grid(points_0124, triangles_0124, pressure_coefs_0124, contour_0124, boundary_0124, res=112, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.3, y_max=0.3) # actual
img = extract_last_image("size/grid_middle_0124.png")
cv2.imwrite("size/extracted_middle.png", img)

In [152]:
## LARGE
plot_pressure_on_grid(points_0013, triangles_0013, pressure_coefs_0013, contour_0013, boundary_0013, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.2, x_max=1.2, y_min=-0.15, y_max=0.15)
plot_black_and_white(points_0013, triangles_0013, pressure_coefs_0013, contour_0013, boundary_0013, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.2, x_max=1.2, y_min=-0.15, y_max=0.15)
plot_black_and_white(points_0124, triangles_0124, pressure_coefs_0124, contour_0124, boundary_0124, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.2, x_max=1.2, y_min=-0.15, y_max=0.15)
plot_black_and_white(points_0124, triangles_0124, pressure_coefs_0124, contour_0124, boundary_0124, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.2, x_max=1.2, y_min=-0.5, y_max=0.5) # MORE FLAT
plot_pressure_on_grid(points_0124, triangles_0124, pressure_coefs_0124, contour_0124, boundary_0124, res=112, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.2, x_max=1.2, y_min=-0.15, y_max=0.15) # actual
plot_pressure_on_grid(points_0124, triangles_0124, pressure_coefs_0124, contour_0124, boundary_0124, res=112, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.2, x_max=1.2, y_min=-0.5, y_max=0.5) # actual

img = extract_last_image("size/grid_large_0124.png")
cv2.imwrite("size/extracted_large.png", img)

img = extract_last_image("size/grid_large_flat_0124.png")
cv2.imwrite("size/extracted_large_flat.png", img)

### Compare Metrics

**Airfoil IoU**

In [162]:
compare_airfoil_overlap('size/extracted_small.png','size/0124_actual_small.png')

(463, 1000, '46.30%')

In [163]:
compare_airfoil_overlap('size/extracted_middle.png','size/0124_actual_middle.png')

(1445, 2000, '72.25%')

In [164]:
compare_airfoil_overlap('size/extracted_large.png','size/0124_actual_large.png')  # significant improvement compared to small

(4767, 5762, '82.73%')

In [181]:
compare_airfoil_overlap('size/extracted_large_flat.png','size/0124_actual_large_flat.png')

(1408, 1870, '75.29%')

**Visual Inspection**

In [167]:
overlay_difference('size/extracted_small.png','size/0124_actual_small.png')

In [168]:
overlay_difference('size/extracted_middle.png','size/0124_actual_middle.png') # quite good

In [169]:
overlay_difference('size/extracted_large.png','size/0124_actual_large.png') # pretty good. airfoil area overlaps a lot.

In [182]:
overlay_difference('size/extracted_large_flat.png','size/0124_actual_large_flat.png') # airfoil overlaps well

**MSE**

In [170]:
compute_mse('size/extracted_small.png','size/0124_actual_small.png')

101.3125

In [171]:
compute_mse('size/extracted_middle.png','size/0124_actual_middle.png')

83.44427614795919

In [172]:
compute_mse('size/extracted_large.png','size/0124_actual_large.png') # decreasing trend

52.15521364795919

In [183]:
compute_mse('size/extracted_large_flat.png','size/0124_actual_large_flat.png')

72.42458545918367

**SSIM**

In [173]:
compute_ssim('size/extracted_small.png','size/0124_actual_small.png')

0.6390489084700608

In [174]:
compute_ssim('size/extracted_middle.png','size/0124_actual_middle.png')

0.6694754059821213

In [175]:
compute_ssim('size/extracted_large.png','size/0124_actual_large.png')  # increasing trend

0.7068277907471062

In [184]:
compute_ssim('size/extracted_large_flat.png','size/0124_actual_large_flat.png') 

0.6418031312788608

**R-squared**

In [176]:
compute_r_squared('size/0124_actual_small.png','size/extracted_small.png')

'0.9661'

In [177]:
compute_r_squared('size/0124_actual_middle.png','size/extracted_middle.png')

'0.9814'

In [178]:
compute_r_squared('size/0124_actual_large.png','size/extracted_large.png')  # increasing trend

'0.9924'

In [185]:
compute_r_squared('size/0124_actual_large_flat.png','size/extracted_large_flat.png') 

'0.9854'

# Does color help?

## Single Version

Using 0013 to predict 0027. The only variation is colored or bw.

In [126]:
plot_pressure_on_grid(points_0013, triangles_0013, pressure_coefs_0013, contour_0013, boundary_0013, res=1000, start_color=WHITE, end_color=BLUE, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.3, y_max=0.3)

plot_black_and_white(points_0013, triangles_0013, pressure_coefs_0013, contour_0013, boundary_0013, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.3, y_max=0.3)

plot_black_and_white(points_0027, triangles_0027, pressure_coefs_0027, contour_0027, boundary_0027, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.3, y_max=0.3)

plot_pressure_on_grid(points_0013, triangles_0013, pressure_coefs_0013, contour_0013, boundary_0013, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.3, y_max=0.3)

plot_pressure_on_grid(points_0027, triangles_0027, pressure_coefs_0027, contour_0027, boundary_0027, res=112, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.3, y_max=0.3)

plot_pressure_on_grid(points_0027, triangles_0027, pressure_coefs_0027, contour_0027, boundary_0027, res=112, start_color=WHITE, end_color=BLUE, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.3, y_max=0.3)

In [128]:
img = extract_last_image("color/grid_bw.png")
cv2.imwrite("color/extracted_bw.png", img)

img = extract_last_image("color/grid_color.png")
cv2.imwrite("color/extracted_color.png", img)

In [581]:
compare_airfoil_overlap('color/extracted_bw.png','color/0027_actual_112_bw.png')

0.7110923660902573

In [582]:
compare_airfoil_overlap('color/extracted_color.png','color/0027_actual_112_color.png')

0.8300198807157058

In [130]:
overlay_difference('color/extracted_bw.png','color/0027_actual_112_bw.png')

In [131]:
overlay_difference('color/extracted_color.png','color/0027_actual_112_color.png')

In [132]:
compute_mse('color/extracted_bw.png','color/0027_actual_112_bw.png')

84.4126275510204

In [133]:
compute_mse('color/extracted_color.png','color/0027_actual_112_color.png')

81.6328125

In [134]:
compute_ssim('color/extracted_bw.png','color/0027_actual_112_bw.png')

0.6401795029776779

In [135]:
compute_ssim('color/extracted_color.png','color/0027_actual_112_color.png')

0.7441673413137828

In [136]:
compute_r_squared('color/0027_actual_112_bw.png', 'color/extracted_bw.png')

'0.9828'

In [137]:
compute_r_squared('color/0027_actual_112_color.png', 'color/extracted_color.png')

'0.9820'

## Loop Version

Use a to predict b. Change color only.

In [514]:
FOLDER1 = 'color/original/color_a/'  # TO CHANGE
FOLDER2 = 'restart_csv/'  # TO CHANGE
data_color_a = read_in_many_files(FOLDER1, FOLDER2)

FOLDER1 = 'color/original/color_b/'  # TO CHANGE
FOLDER2 = 'restart_csv/'  # TO CHANGE
data_color_b = read_in_many_files(FOLDER1, FOLDER2)

### Black and White Version

In [522]:
# 右上
plot_presure_on_grid_LOOP(data_color_a, 500, WHITE, BLACK, BLACK, -0.4, 1.4, -0.3, 0.3, 'color/ur/black_and_white')

# 左上  
plot_black_and_white_LOOP(data_color_a, 500, BLACK, WHITE, -0.4, 1.4, -0.3, 0.3, 'color/ul')

# 左下
plot_black_and_white_LOOP(data_color_b, 500, BLACK, WHITE, -0.4, 1.4, -0.3, 0.3, 'color/ll')

input_folder = 'color/generated_grid/black_and_white'
output_folder = 'color/extracted/black_and_white'
extract_last_image_LOOP(input_folder, output_folder)

# 右下
plot_presure_on_grid_LOOP(data_color_b, 112, WHITE, BLACK, BLACK, -0.4, 1.4, -0.3, 0.3, 'color/lr/black_and_white')

### Color Version (Blue)

In [523]:
# 右上
plot_presure_on_grid_LOOP(data_color_a, 500, WHITE, BLUE, BLACK, -0.4, 1.4, -0.3, 0.3, 'color/ur/color')

input_folder = 'color/generated_grid/color'
output_folder = 'color/extracted/color'
extract_last_image_LOOP(input_folder, output_folder)

# 右下
plot_presure_on_grid_LOOP(data_color_b, 112, WHITE, BLUE, BLACK, -0.4, 1.4, -0.3, 0.3, 'color/lr/color')

### Color Version (Red)

In [539]:
# 右上
plot_presure_on_grid_LOOP(data_color_a, 500, WHITE, RED, BLACK, -0.4, 1.4, -0.3, 0.3, 'color/ur/red')

input_folder = 'color/generated_grid/red'
output_folder = 'color/extracted/red'
extract_last_image_LOOP(input_folder, output_folder)

# 右下
plot_presure_on_grid_LOOP(data_color_b, 112, WHITE, RED, BLACK, -0.4, 1.4, -0.3, 0.3, 'color/lr/red')

### Color Version (Yellow)

In [549]:
# 右上
plot_presure_on_grid_LOOP(data_color_a, 500, WHITE, YELLOW, BLACK, -0.4, 1.4, -0.3, 0.3, 'color/ur/yellow')

input_folder = 'color/generated_grid/yellow'
output_folder = 'color/extracted/yellow'
extract_last_image_LOOP(input_folder, output_folder)

# 右下
plot_presure_on_grid_LOOP(data_color_b, 112, WHITE, YELLOW, BLACK, -0.4, 1.4, -0.3, 0.3, 'color/lr/yellow')

### Color Version (Green)

In [550]:
# 右上
plot_presure_on_grid_LOOP(data_color_a, 500, WHITE, GREEN, BLACK, -0.4, 1.4, -0.3, 0.3, 'color/ur/green')

input_folder = 'color/generated_grid/green'
output_folder = 'color/extracted/green'
extract_last_image_LOOP(input_folder, output_folder)

# 右下
plot_presure_on_grid_LOOP(data_color_b, 112, WHITE, GREEN, BLACK, -0.4, 1.4, -0.3, 0.3, 'color/lr/green')

### Compare Metrics

#### Airfoil overlap

In [531]:
# BLUE
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
iou_bw = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/color'
actual_folder = 'color/lr/color'
iou_color = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

diff = iou_color - iou_bw
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of positive increase from bw to color: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(iou_bw, iou_color)
print("t-statistic for bw and color: {:.4f}".format(t_statistic))
print("p-value for bw and color: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("iou_bw:\n", iou_bw)
print("iou_color:\n", iou_color)

Proportion of positive increase from bw to color: 84.00%
t-statistic for bw and color: -3.8249
p-value for bw and color: 0.0008
SIGNIFICANT!

iou_bw:
 [0.66443515 0.61127273 0.71179951 0.73471539 0.72584856 0.70691676
 0.6202765  0.63435583 0.76476907 0.79495942 0.67222767 0.75107632
 0.715      0.62236842 0.69430052 0.69724409 0.58797041 0.74668142
 0.51238165 0.52078337 0.70872274 0.56637737 0.75848913 0.72397869
 0.78151862]
iou_color:
 [0.69881711 0.66342649 0.71706904 0.75494071 0.7436182  0.7591267
 0.61171797 0.61766024 0.73103448 0.83514407 0.69048474 0.7726087
 0.81442495 0.70567195 0.78153697 0.76740009 0.65962264 0.80735478
 0.76616915 0.53912257 0.72064601 0.60899131 0.79143228 0.69575758
 0.85720199]


In [552]:
# RED
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
iou_bw = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/red'
actual_folder = 'color/lr/red'
iou_red = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

diff = iou_red - iou_bw
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of positive increase from bw to red: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(iou_bw, iou_red)
print("t-statistic for bw and red: {:.4f}".format(t_statistic))
print("p-value for bw and red: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("iou_bw:\n", iou_bw)
print("iou_red:\n", iou_red)

Proportion of positive increase from bw to red: 96.00%
t-statistic for bw and red: -6.1465
p-value for bw and red: 0.0000
SIGNIFICANT!

iou_bw:
 [0.66443515 0.61127273 0.71179951 0.73471539 0.72584856 0.70691676
 0.6202765  0.63435583 0.76476907 0.79495942 0.67222767 0.75107632
 0.715      0.62236842 0.69430052 0.69724409 0.58797041 0.74668142
 0.51238165 0.52078337 0.70872274 0.56637737 0.75848913 0.72397869
 0.78151862]
iou_red:
 [0.78365385 0.67235637 0.75555556 0.77559549 0.78170144 0.77739726
 0.64488017 0.64178381 0.77210884 0.83014354 0.7143951  0.76972184
 0.80887097 0.80750894 0.80583942 0.76961909 0.68057722 0.80232058
 0.74015411 0.57122093 0.73846843 0.63760751 0.79920036 0.71803415
 0.8716303 ]


In [553]:
# YELLOW
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
iou_bw = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/yellow'
actual_folder = 'color/lr/yellow'
iou_yellow = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

diff = iou_yellow - iou_bw
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of positive increase from bw to yellow: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(iou_bw, iou_yellow)
print("t-statistic for bw and yellow: {:.4f}".format(t_statistic))
print("p-value for bw and yellow: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("iou_bw:\n", iou_bw)
print("iou_yellow:\n", iou_yellow)

Proportion of positive increase from bw to yellow: 96.00%
t-statistic for bw and yellow: -7.0184
p-value for bw and yellow: 0.0000
SIGNIFICANT!

iou_bw:
 [0.66443515 0.61127273 0.71179951 0.73471539 0.72584856 0.70691676
 0.6202765  0.63435583 0.76476907 0.79495942 0.67222767 0.75107632
 0.715      0.62236842 0.69430052 0.69724409 0.58797041 0.74668142
 0.51238165 0.52078337 0.70872274 0.56637737 0.75848913 0.72397869
 0.78151862]
iou_yellow:
 [0.75012494 0.72957111 0.75954023 0.80769231 0.78716356 0.80487805
 0.67128603 0.62573964 0.83435961 0.87118469 0.76803279 0.75729572
 0.82419899 0.80012771 0.85942492 0.75963827 0.68122638 0.83333333
 0.78011644 0.58714703 0.74267101 0.62955466 0.80129991 0.72825617
 0.87302305]


In [554]:
# GREEN
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
iou_bw = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/green'
actual_folder = 'color/lr/green'
iou_green = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

diff = iou_green - iou_bw
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of positive increase from bw to green: {:.2f}%".format(increase))  # iou效果非常好

t_statistic, p_value = stats.ttest_rel(iou_bw, iou_green)
print("t-statistic for bw and green: {:.4f}".format(t_statistic))
print("p-value for bw and green: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("iou_bw:\n", iou_bw)
print("iou_green:\n", iou_green)

Proportion of positive increase from bw to green: 100.00%
t-statistic for bw and green: -9.6985
p-value for bw and green: 0.0000
SIGNIFICANT!

iou_bw:
 [0.66443515 0.61127273 0.71179951 0.73471539 0.72584856 0.70691676
 0.6202765  0.63435583 0.76476907 0.79495942 0.67222767 0.75107632
 0.715      0.62236842 0.69430052 0.69724409 0.58797041 0.74668142
 0.51238165 0.52078337 0.70872274 0.56637737 0.75848913 0.72397869
 0.78151862]
iou_green:
 [0.82959641 0.7325529  0.77134986 0.81241915 0.80795778 0.77933384
 0.70005414 0.6781721  0.83716707 0.8974359  0.74633373 0.78362573
 0.81132861 0.82569361 0.82637475 0.79942693 0.71696574 0.84009009
 0.78565102 0.6269305  0.78877128 0.67169657 0.83908582 0.76015727
 0.88379341]


In [583]:
# BLUE AND RED: RED BETTER
diff = iou_red - iou_color
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of positive increase from blue to red: {:.2f}%".format(increase))  # iou效果非常好

t_statistic, p_value = stats.ttest_rel(iou_color, iou_red)
print("t-statistic for blue and red: {:.4f}".format(t_statistic))
print("p-value for blue and red: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("iou_color:\n", iou_color)
print("iou_red:\n", iou_red)

Proportion of positive increase from blue to red: 80.00%
t-statistic for blue and red: -4.1433
p-value for blue and red: 0.0004
SIGNIFICANT!

iou_color:
 [0.69881711 0.66342649 0.71706904 0.75494071 0.7436182  0.7591267
 0.61171797 0.61766024 0.73103448 0.83514407 0.69048474 0.7726087
 0.81442495 0.70567195 0.78153697 0.76740009 0.65962264 0.80735478
 0.76616915 0.53912257 0.72064601 0.60899131 0.79143228 0.69575758
 0.85720199]
iou_red:
 [0.78365385 0.67235637 0.75555556 0.77559549 0.78170144 0.77739726
 0.64488017 0.64178381 0.77210884 0.83014354 0.7143951  0.76972184
 0.80887097 0.80750894 0.80583942 0.76961909 0.68057722 0.80232058
 0.74015411 0.57122093 0.73846843 0.63760751 0.79920036 0.71803415
 0.8716303 ]


In [571]:
# RED AND YELLOW：YELLOW BETTER
diff = iou_yellow - iou_red
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of positive increase from red to yellow: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(iou_red, iou_yellow)
print("t-statistic for red and yellow: {:.4f}".format(t_statistic))
print("p-value for red and yellow: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("iou_red:\n", iou_red)
print("iou_yellow:\n", iou_yellow)

Proportion of positive increase from red to yellow: 76.00%
t-statistic for red and yellow: -3.0957
p-value for red and yellow: 0.0049
SIGNIFICANT!

iou_red:
 [0.78365385 0.67235637 0.75555556 0.77559549 0.78170144 0.77739726
 0.64488017 0.64178381 0.77210884 0.83014354 0.7143951  0.76972184
 0.80887097 0.80750894 0.80583942 0.76961909 0.68057722 0.80232058
 0.74015411 0.57122093 0.73846843 0.63760751 0.79920036 0.71803415
 0.8716303 ]
iou_yellow:
 [0.75012494 0.72957111 0.75954023 0.80769231 0.78716356 0.80487805
 0.67128603 0.62573964 0.83435961 0.87118469 0.76803279 0.75729572
 0.82419899 0.80012771 0.85942492 0.75963827 0.68122638 0.83333333
 0.78011644 0.58714703 0.74267101 0.62955466 0.80129991 0.72825617
 0.87302305]


In [573]:
# GREEN AND YELLOW: green larger (better)
diff = iou_yellow - iou_green
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of positive increase from green to yellow: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(iou_green, iou_yellow)
print("t-statistic for green and yellow: {:.4f}".format(t_statistic))
print("p-value for green and yellow: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("iou_green:\n", iou_green)
print("iou_yellow:\n", iou_yellow)

Proportion of positive increase from green to yellow: 16.00%
t-statistic for green and yellow: 3.7011
p-value for green and yellow: 0.0011
SIGNIFICANT!

iou_green:
 [0.82959641 0.7325529  0.77134986 0.81241915 0.80795778 0.77933384
 0.70005414 0.6781721  0.83716707 0.8974359  0.74633373 0.78362573
 0.81132861 0.82569361 0.82637475 0.79942693 0.71696574 0.84009009
 0.78565102 0.6269305  0.78877128 0.67169657 0.83908582 0.76015727
 0.88379341]
iou_yellow:
 [0.75012494 0.72957111 0.75954023 0.80769231 0.78716356 0.80487805
 0.67128603 0.62573964 0.83435961 0.87118469 0.76803279 0.75729572
 0.82419899 0.80012771 0.85942492 0.75963827 0.68122638 0.83333333
 0.78011644 0.58714703 0.74267101 0.62955466 0.80129991 0.72825617
 0.87302305]


#### MSE

In [533]:
# BLUE
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
mse_bw = compute_mse_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/color'
actual_folder = 'color/lr/color'
mse_color = compute_mse_LOOP(predicted_folder, actual_folder)

diff = mse_color - mse_bw
decrease = np.sum(diff < 0) / len(diff) * 100
print("Proportion of decrease from bw to color: {:.2f}%".format(decrease))

t_statistic, p_value = stats.ttest_rel(mse_bw, mse_color)
print("t-statistic for bw and color: {:.4f}".format(t_statistic))
print("p-value for bw and color: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("mse_bw:\n", mse_bw)
print("mse_color:\n", mse_color)

Proportion of decrease from bw to color: 88.00%
t-statistic for bw and color: 5.0574
p-value for bw and color: 0.0000
SIGNIFICANT!

mse_bw:
 [78.70296556 36.30245536 22.96109694 32.17370855 40.18821747 46.0444037
 67.72951212 64.9190051  40.1661352  36.57581314 35.34789541 63.75255102
 38.61128827 64.63942921 55.82940051 41.40425702 50.81648597 37.98134566
 63.1558514  71.03188776 42.46420599 60.75813138 33.35586735 39.29567921
 33.9740912 ]
mse_color:
 [48.4634088  27.18423151 35.11981824 33.29121492 25.8330676  27.08673469
 52.68662309 39.17195472 24.62404337 30.50143495 34.15617028 32.14827806
 32.37619579 36.0020727  22.7635523  40.63990753 40.26929209 38.22441008
 35.71029974 52.20304528 35.81831952 51.39564732 33.2574139  32.49944196
 31.22472895]


In [557]:
# RED
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
mse_bw = compute_mse_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/red'
actual_folder = 'color/lr/red'
mse_red = compute_mse_LOOP(predicted_folder, actual_folder)

diff = mse_red - mse_bw
decrease = np.sum(diff < 0) / len(diff) * 100
print("Proportion of decrease from bw to red: {:.2f}%".format(decrease))

t_statistic, p_value = stats.ttest_rel(mse_bw, mse_red)
print("t-statistic for bw and red: {:.4f}".format(t_statistic))
print("p-value for bw and red: {:.4f}".format(p_value))
print("SIGNIFICANT!")
print("BW VERSION IS BETTER\n")

print("mse_bw:\n", mse_bw)
print("mse_red:\n", mse_red)

Proportion of decrease from bw to red: 24.00%
t-statistic for bw and red: -2.2113
p-value for bw and red: 0.0368
SIGNIFICANT!
BW VERSION IS BETTER

mse_bw:
 [78.70296556 36.30245536 22.96109694 32.17370855 40.18821747 46.0444037
 67.72951212 64.9190051  40.1661352  36.57581314 35.34789541 63.75255102
 38.61128827 64.63942921 55.82940051 41.40425702 50.81648597 37.98134566
 63.1558514  71.03188776 42.46420599 60.75813138 33.35586735 39.29567921
 33.9740912 ]
mse_red:
 [79.16573661 38.69754464 31.40058992 48.8125     47.97018495 50.74362245
 79.95192921 71.44196429 45.13871173 31.62858737 45.14508929 67.94977679
 52.91501913 44.71316964 43.30125957 45.64867666 56.47177934 43.20248724
 71.23732462 69.96037946 40.45838648 59.49641263 35.7964764  43.00693559
 40.15967793]


In [561]:
# YELLOW
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
mse_bw = compute_mse_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/yellow'
actual_folder = 'color/lr/yellow'
mse_yellow = compute_mse_LOOP(predicted_folder, actual_folder)

diff = mse_yellow - mse_bw
decrease = np.sum(diff < 0) / len(diff) * 100
print("Proportion of decrease from bw to yellow: {:.2f}%".format(decrease))

t_statistic, p_value = stats.ttest_rel(mse_bw, mse_yellow)                    # 黄色效果非常好
print("t-statistic for bw and yellow: {:.4f}".format(t_statistic))
print("p-value for bw and yellow: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("mse_bw:\n", mse_bw)
print("mse_yellow:\n", mse_yellow)

Proportion of decrease from bw to yellow: 100.00%
t-statistic for bw and yellow: 12.1566
p-value for bw and yellow: 0.0000
SIGNIFICANT!

mse_bw:
 [78.70296556 36.30245536 22.96109694 32.17370855 40.18821747 46.0444037
 67.72951212 64.9190051  40.1661352  36.57581314 35.34789541 63.75255102
 38.61128827 64.63942921 55.82940051 41.40425702 50.81648597 37.98134566
 63.1558514  71.03188776 42.46420599 60.75813138 33.35586735 39.29567921
 33.9740912 ]
mse_yellow:
 [ 6.54360651 11.71795281 14.24736926 11.85068559 11.26713967 12.3881537
  9.81927615 15.06752232 12.52032844  6.66557717 11.68231824 21.3934949
  9.40888074 13.67235332 15.56871811 16.39827806 22.23413584  9.2290338
 13.82557398 14.22496811  9.41334503 12.29280931  7.8934949  10.59773597
 12.05731824]


In [562]:
# GREEN
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
mse_bw = compute_mse_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/green'
actual_folder = 'color/lr/green'
mse_green = compute_mse_LOOP(predicted_folder, actual_folder)

diff = mse_green - mse_bw
decrease = np.sum(diff < 0) / len(diff) * 100
print("Proportion of decrease from bw to green: {:.2f}%".format(decrease))

t_statistic, p_value = stats.ttest_rel(mse_bw, mse_green)                    # 绿色效果也不错
print("t-statistic for bw and green: {:.4f}".format(t_statistic))
print("p-value for bw and green: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("mse_bw:\n", mse_bw)
print("mse_green:\n", mse_green)

Proportion of decrease from bw to green: 80.00%
t-statistic for bw and green: 4.2700
p-value for bw and green: 0.0003
SIGNIFICANT!

mse_bw:
 [78.70296556 36.30245536 22.96109694 32.17370855 40.18821747 46.0444037
 67.72951212 64.9190051  40.1661352  36.57581314 35.34789541 63.75255102
 38.61128827 64.63942921 55.82940051 41.40425702 50.81648597 37.98134566
 63.1558514  71.03188776 42.46420599 60.75813138 33.35586735 39.29567921
 33.9740912 ]
mse_green:
 [35.09255421 33.82453763 28.37053571 29.17904974 36.86208546 35.15553253
 37.52192283 15.32828444  7.07653061 37.6278699  29.19961735 25.83641582
 38.99481824 19.16358418 44.09941008 39.05923151 31.11033163 29.02000957
 46.8178412  37.51753827 37.61918048 43.03627232 40.36583227 34.3227838
 38.38137755]


In [574]:
# BLUE AND RED: BLUE GOOD
diff = mse_color - mse_red
decrease = np.sum(diff < 0) / len(diff) * 100
print("Proportion of decrease from red to blue: {:.2f}%".format(decrease))

t_statistic, p_value = stats.ttest_rel(mse_red, mse_color)
print("t-statistic for red and blue: {:.4f}".format(t_statistic))
print("p-value for red and blue: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("mse_red:\n", mse_red)
print("mse_blue:\n", mse_color)

Proportion of decrease from red to blue: 96.00%
t-statistic for red and blue: 7.0755
p-value for red and blue: 0.0000
SIGNIFICANT!

mse_red:
 [79.16573661 38.69754464 31.40058992 48.8125     47.97018495 50.74362245
 79.95192921 71.44196429 45.13871173 31.62858737 45.14508929 67.94977679
 52.91501913 44.71316964 43.30125957 45.64867666 56.47177934 43.20248724
 71.23732462 69.96037946 40.45838648 59.49641263 35.7964764  43.00693559
 40.15967793]
mse_blue:
 [48.4634088  27.18423151 35.11981824 33.29121492 25.8330676  27.08673469
 52.68662309 39.17195472 24.62404337 30.50143495 34.15617028 32.14827806
 32.37619579 36.0020727  22.7635523  40.63990753 40.26929209 38.22441008
 35.71029974 52.20304528 35.81831952 51.39564732 33.2574139  32.49944196
 31.22472895]


In [575]:
# BLUE AND YELLOW: YELLOW BEST
diff = mse_yellow - mse_color
decrease = np.sum(diff < 0) / len(diff) * 100
print("Proportion of decrease from blue to yellow: {:.2f}%".format(decrease))

t_statistic, p_value = stats.ttest_rel(mse_yellow, mse_color)
print("t-statistic for yellow and blue: {:.4f}".format(t_statistic))
print("p-value for yellow and blue: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("mse_yellow:\n", mse_yellow)
print("mse_blue:\n", mse_color)

Proportion of decrease from blue to yellow: 100.00%
t-statistic for yellow and blue: -12.5463
p-value for yellow and blue: 0.0000
SIGNIFICANT!

mse_yellow:
 [ 6.54360651 11.71795281 14.24736926 11.85068559 11.26713967 12.3881537
  9.81927615 15.06752232 12.52032844  6.66557717 11.68231824 21.3934949
  9.40888074 13.67235332 15.56871811 16.39827806 22.23413584  9.2290338
 13.82557398 14.22496811  9.41334503 12.29280931  7.8934949  10.59773597
 12.05731824]
mse_blue:
 [48.4634088  27.18423151 35.11981824 33.29121492 25.8330676  27.08673469
 52.68662309 39.17195472 24.62404337 30.50143495 34.15617028 32.14827806
 32.37619579 36.0020727  22.7635523  40.63990753 40.26929209 38.22441008
 35.71029974 52.20304528 35.81831952 51.39564732 33.2574139  32.49944196
 31.22472895]


In [576]:
# GREEN AND YELLOW: YELLOW BEST
diff = mse_yellow - mse_green
decrease = np.sum(diff < 0) / len(diff) * 100
print("Proportion of decrease from green to yellow: {:.2f}%".format(decrease))

t_statistic, p_value = stats.ttest_rel(mse_yellow, mse_green)
print("t-statistic for yellow and green: {:.4f}".format(t_statistic))
print("p-value for yellow and green: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("mse_yellow:\n", mse_yellow)
print("mse_green:\n", mse_green)

Proportion of decrease from green to yellow: 96.00%
t-statistic for yellow and green: -9.8027
p-value for yellow and green: 0.0000
SIGNIFICANT!

mse_yellow:
 [ 6.54360651 11.71795281 14.24736926 11.85068559 11.26713967 12.3881537
  9.81927615 15.06752232 12.52032844  6.66557717 11.68231824 21.3934949
  9.40888074 13.67235332 15.56871811 16.39827806 22.23413584  9.2290338
 13.82557398 14.22496811  9.41334503 12.29280931  7.8934949  10.59773597
 12.05731824]
mse_green:
 [35.09255421 33.82453763 28.37053571 29.17904974 36.86208546 35.15553253
 37.52192283 15.32828444  7.07653061 37.6278699  29.19961735 25.83641582
 38.99481824 19.16358418 44.09941008 39.05923151 31.11033163 29.02000957
 46.8178412  37.51753827 37.61918048 43.03627232 40.36583227 34.3227838
 38.38137755]


#### R^2

In [536]:
# BLUE
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
r_bw = compute_r_squared_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/color'
actual_folder = 'color/lr/color'
r_color = compute_r_squared_LOOP(predicted_folder, actual_folder)

diff = r_color - r_bw
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of increase from bw to color: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(r_bw, r_color)
print("t-statistic for bw and color: {:.4f}".format(t_statistic))
print("p-value for bw and color: {:.4f}".format(p_value))
print("NOT SIGNIFICANT!\n")

print("R^2_bw:\n", r_bw)
print("R^2_color:\n", r_color)

Proportion of increase from bw to color: 28.00%
t-statistic for bw and color: 1.7332
p-value for bw and color: 0.0959
NOT SIGNIFICANT!

R^2_bw:
 [0.98414701 0.98791296 0.99170938 0.99031848 0.99164113 0.99104383
 0.9875777  0.98855968 0.98678474 0.99035749 0.99278927 0.98857069
 0.99135016 0.98624656 0.98911204 0.98891832 0.98917258 0.98782763
 0.98859946 0.98784074 0.99196617 0.98803847 0.99171165 0.99161611
 0.99152008]
R^2_color:
 [0.98805718 0.98519133 0.98834876 0.98778155 0.98907172 0.99009536
 0.98733004 0.99136982 0.98866372 0.98900366 0.98921039 0.99177598
 0.98805428 0.98766987 0.98826332 0.98813848 0.98887193 0.98726067
 0.98790955 0.98876577 0.9898538  0.98865528 0.98881642 0.98987431
 0.9886846 ]


In [544]:
# RED
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
r_bw = compute_r_squared_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/red'
actual_folder = 'color/lr/red'
r_red = compute_r_squared_LOOP(predicted_folder, actual_folder)

diff = r_red - r_bw
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of increase from bw to red: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(r_bw, r_red)
print("t-statistic for bw and red: {:.4f}".format(t_statistic))
print("p-value for bw and red: {:.4f}".format(p_value))
print("NOT SIGNIFICANT!\n")

print("R^2_bw:\n", r_bw)
print("R^2_red:\n", r_red)

Proportion of increase from bw to red: 44.00%
t-statistic for bw and red: 1.5043
p-value for bw and red: 0.1456
NOT SIGNIFICANT!

R^2_bw:
 [0.98414701 0.98791296 0.99170938 0.99031848 0.99164113 0.99104383
 0.9875777  0.98855968 0.98678474 0.99035749 0.99278927 0.98857069
 0.99135016 0.98624656 0.98911204 0.98891832 0.98917258 0.98782763
 0.98859946 0.98784074 0.99196617 0.98803847 0.99171165 0.99161611
 0.99152008]
R^2_red:
 [0.98168225 0.98830363 0.99212453 0.98892835 0.98996809 0.99019326
 0.98410663 0.9866273  0.98700884 0.99116795 0.98989865 0.9866429
 0.99028129 0.98767627 0.98878035 0.99025018 0.98996971 0.99068486
 0.98675066 0.9865767  0.99118029 0.98931313 0.99223369 0.99198476
 0.99144538]


In [565]:
# YELLOW
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
r_bw = compute_r_squared_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/yellow'
actual_folder = 'color/lr/yellow'
r_yellow = compute_r_squared_LOOP(predicted_folder, actual_folder)

diff = r_yellow - r_bw
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of increase from bw to yellow: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(r_bw, r_yellow)                    # 黄色效果非常好
print("t-statistic for bw and yellow: {:.4f}".format(t_statistic))
print("p-value for bw and yellow: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("R^2_bw:\n", r_bw)
print("R^2_yellow:\n", r_yellow)

Proportion of increase from bw to yellow: 100.00%
t-statistic for bw and yellow: -19.7390
p-value for bw and yellow: 0.0000
SIGNIFICANT!

R^2_bw:
 [0.98414701 0.98791296 0.99170938 0.99031848 0.99164113 0.99104383
 0.9875777  0.98855968 0.98678474 0.99035749 0.99278927 0.98857069
 0.99135016 0.98624656 0.98911204 0.98891832 0.98917258 0.98782763
 0.98859946 0.98784074 0.99196617 0.98803847 0.99171165 0.99161611
 0.99152008]
R^2_yellow:
 [0.99667734 0.99633123 0.99718902 0.99696921 0.99718209 0.99748532
 0.99710512 0.9970804  0.99680503 0.99696114 0.99758815 0.9967562
 0.99757375 0.99662054 0.99702464 0.9973615  0.99714903 0.99794246
 0.99754321 0.9967666  0.99785248 0.99755771 0.99723026 0.99738905
 0.99731607]


In [566]:
# GREEN
predicted_folder = 'color/extracted/black_and_white'
actual_folder = 'color/lr/black_and_white'
r_bw = compute_r_squared_LOOP(predicted_folder, actual_folder)

predicted_folder = 'color/extracted/green'
actual_folder = 'color/lr/green'
r_green = compute_r_squared_LOOP(predicted_folder, actual_folder)

diff = r_green - r_bw
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of increase from bw to green: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(r_bw, r_green)                  # 绿色效果还可以 
print("t-statistic for bw and green: {:.4f}".format(t_statistic))
print("p-value for bw and green: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("R^2_bw:\n", r_bw)
print("R^2_green:\n", r_green)

Proportion of increase from bw to green: 60.00%
t-statistic for bw and green: -3.2752
p-value for bw and green: 0.0032
SIGNIFICANT!

R^2_bw:
 [0.98414701 0.98791296 0.99170938 0.99031848 0.99164113 0.99104383
 0.9875777  0.98855968 0.98678474 0.99035749 0.99278927 0.98857069
 0.99135016 0.98624656 0.98911204 0.98891832 0.98917258 0.98782763
 0.98859946 0.98784074 0.99196617 0.98803847 0.99171165 0.99161611
 0.99152008]
R^2_green:
 [0.98831065 0.98935549 0.99165092 0.99028574 0.9900718  0.99074628
 0.98974143 0.99322682 0.98882632 0.98999305 0.99092364 0.99301322
 0.99211092 0.99065462 0.98754147 0.9910368  0.99358684 0.991954
 0.99004913 0.99088393 0.99127326 0.99203906 0.99131917 0.992581
 0.99035851]


In [577]:
# BLUE AND RED: 差不多
diff = r_color - r_red
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of increase from red to blue: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(r_red, r_color)
print("t-statistic for red and blue: {:.4f}".format(t_statistic))
print("p-value for red and blue: {:.4f}".format(p_value))
print("NOT SIGNIFICANT!\n")

print("R^2_red:\n", r_red)
print("R^2_blue:\n", r_color)

Proportion of increase from red to blue: 28.00%
t-statistic for red and blue: 0.5066
p-value for bw and blue: 0.6171
NOT SIGNIFICANT!

R^2_red:
 [0.98168225 0.98830363 0.99212453 0.98892835 0.98996809 0.99019326
 0.98410663 0.9866273  0.98700884 0.99116795 0.98989865 0.9866429
 0.99028129 0.98767627 0.98878035 0.99025018 0.98996971 0.99068486
 0.98675066 0.9865767  0.99118029 0.98931313 0.99223369 0.99198476
 0.99144538]
R^2_blue:
 [0.98805718 0.98519133 0.98834876 0.98778155 0.98907172 0.99009536
 0.98733004 0.99136982 0.98866372 0.98900366 0.98921039 0.99177598
 0.98805428 0.98766987 0.98826332 0.98813848 0.98887193 0.98726067
 0.98790955 0.98876577 0.9898538  0.98865528 0.98881642 0.98987431
 0.9886846 ]


In [584]:
# BLUE AND YELLOW: YELLOW BEST
diff = r_yellow - r_color
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of increase from blue to yellow: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(r_color, r_yellow)
print("t-statistic for blue and yellow: {:.4f}".format(t_statistic))
print("p-value for blue and yellow: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("R^2_blue:\n", r_color)
print("R^2_yellow:\n", r_yellow)

Proportion of increase from blue to yellow: 100.00%
t-statistic for blue and yellow: -32.6433
p-value for blue and yellow: 0.0000
SIGNIFICANT!

R^2_blue:
 [0.98805718 0.98519133 0.98834876 0.98778155 0.98907172 0.99009536
 0.98733004 0.99136982 0.98866372 0.98900366 0.98921039 0.99177598
 0.98805428 0.98766987 0.98826332 0.98813848 0.98887193 0.98726067
 0.98790955 0.98876577 0.9898538  0.98865528 0.98881642 0.98987431
 0.9886846 ]
R^2_yellow:
 [0.99667734 0.99633123 0.99718902 0.99696921 0.99718209 0.99748532
 0.99710512 0.9970804  0.99680503 0.99696114 0.99758815 0.9967562
 0.99757375 0.99662054 0.99702464 0.9973615  0.99714903 0.99794246
 0.99754321 0.9967666  0.99785248 0.99755771 0.99723026 0.99738905
 0.99731607]


In [585]:
# GREEN AND YELLOW: YELLOW BEST
diff = r_yellow - r_green
increase = np.sum(diff > 0) / len(diff) * 100
print("Proportion of increase from green to yellow: {:.2f}%".format(increase))

t_statistic, p_value = stats.ttest_rel(r_green, r_yellow)
print("t-statistic for green and yellow: {:.4f}".format(t_statistic))
print("p-value for green and yellow: {:.4f}".format(p_value))
print("SIGNIFICANT!\n")

print("R^2_green:\n", r_green)
print("R^2_yellow:\n", r_yellow)

Proportion of increase from green to yellow: 100.00%
t-statistic for green and yellow: -22.5735
p-value for green and yellow: 0.0000
SIGNIFICANT!

R^2_green:
 [0.98831065 0.98935549 0.99165092 0.99028574 0.9900718  0.99074628
 0.98974143 0.99322682 0.98882632 0.98999305 0.99092364 0.99301322
 0.99211092 0.99065462 0.98754147 0.9910368  0.99358684 0.991954
 0.99004913 0.99088393 0.99127326 0.99203906 0.99131917 0.992581
 0.99035851]
R^2_yellow:
 [0.99667734 0.99633123 0.99718902 0.99696921 0.99718209 0.99748532
 0.99710512 0.9970804  0.99680503 0.99696114 0.99758815 0.9967562
 0.99757375 0.99662054 0.99702464 0.9973615  0.99714903 0.99794246
 0.99754321 0.9967666  0.99785248 0.99755771 0.99723026 0.99738905
 0.99731607]


# Compare: 1. Using B's coordinates and A's pressure   2. Generated B from model

**Example: A: 0719; B:0013. Use 0719 to predict 0013**

In [186]:
plot_pressure_on_grid(points_0013, triangles_0013, pressure_coefs_0013, contour_0013, boundary_0013, res=112, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.3, y_max=0.3)

In [188]:
compare_airfoil_overlap('compare_a_b/extracted_0013.png','compare_a_b/0013_actual_112.png')

(835, 1236, '67.56%')

In [189]:
compare_airfoil_overlap('compare_a_b/0013_with_0719_pressure.png','compare_a_b/0013_actual_112.png')

(863, 901, '95.78%')

In [201]:
overlay_difference('compare_a_b/extracted_0013.png','compare_a_b/0013_actual_112.png')

In [202]:
overlay_difference('compare_a_b/0013_with_0719_pressure.png','compare_a_b/0013_actual_112.png')

In [191]:
compute_ssim('compare_a_b/extracted_0013.png','compare_a_b/0013_actual_112.png')

0.7213416091155113

In [210]:
compute_mse('compare_a_b/extracted_0013.png','compare_a_b/0013_actual_112.png')

30.612882653061224

In [209]:
compute_mse('compare_a_b/0013_with_0719_pressure.png','compare_a_b/0013_actual_112.png')

5.53515625

In [192]:
compute_ssim('compare_a_b/0013_with_0719_pressure.png','compare_a_b/0013_actual_112.png')

0.8765252969260813

In [193]:
compute_r_squared('compare_a_b/0013_actual_112.png', 'compare_a_b/extracted_0013.png')

'0.9817'

In [194]:
compute_r_squared('compare_a_b/0013_actual_112.png', 'compare_a_b/0013_with_0719_pressure.png')

'0.9724'

# Aggregation Task

## Loop Version

Use airfoil 0027 as main subject. Choose 50 airfoils for using model. Use the same 50 airfoils for coordinates-pressure replacement. Compare result.

1. Using Model

In [427]:
FOLDER1 = 'aggregation/original/'  # TO CHANGE
FOLDER2 = 'restart_csv/'  # TO CHANGE
data = read_in_many_files(FOLDER1, FOLDER2)

In [434]:
# 右上
plot_presure_on_grid_LOOP(data, 500, WHITE, BLACK, BLACK, -0.4, 1.4, -0.3, 0.3, 'aggregation/actual')
# 左上
plot_black_and_white_LOOP(data, 500, BLACK, WHITE, -0.4, 1.4, -0.3, 0.3, 'aggregation/actual_bw')
# 左下
plot_black_and_white(points_0027, triangles_0027, pressure_coefs_0027, contour_0027, boundary_0027, res=500, inside_color=BLACK, outside_color=WHITE, x_min=-0.4, x_max=1.4, y_min=-0.3, y_max=0.3) 

In [439]:
# use model
input_folder = 'aggregation/generated_grid'
output_folder = 'aggregation/extracted'
extract_last_image_LOOP(input_folder, output_folder)

# 右下
plot_pressure_on_grid(points_0027, triangles_0027, pressure_coefs_0027, contour_0027, boundary_0027, res=112, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.4, x_max=1.4, y_min=-0.3, y_max=0.3)

In [441]:
def aggregate_images_mean(folder_path):
    
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.png')]
    image_sum = None
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image_sum is None:
            image_sum = np.zeros_like(image, dtype=np.float64)
        image_sum += image
    
    mean_image = image_sum / len(image_files)
    mean_image = np.clip(mean_image, 0, 255).astype(np.uint8)
    
    return mean_image

In [442]:
folder_path = 'aggregation/extracted'
mean_image = aggregate_images_mean(folder_path)
cv2.imwrite('aggregation/mean_image_from_model_LOOP.png', mean_image)

True

2. Use 0027's coordinates and other airfoils' pressure

In [454]:
def read_in_many_files_2(FOLDER1, FOLDER2):
    files_in_folder_1 = os.listdir(FOLDER1)
    data = {}
    for file1 in files_in_folder_1:
        identifier = get_identifier(file1)
        file2 = str(identifier) + '.csv'
        if file2 in os.listdir(FOLDER2):
            points, triangles, contour, boundary, pressure_coefs = read_all_data(file1, file2)
        # store in dictionary
        data[identifier] = {  
                'points': points_0027,
                'triangles': triangles_0027,
                'contour': contour_0027,
                'boundary': boundary_0027,
                'pressure_coefs': pressure_coefs}
    return data

FOLDER1 = 'aggregation/original/'  # TO CHANGE
FOLDER2 = 'restart_csv/'  # TO CHANGE
data = read_in_many_files_2(FOLDER1, FOLDER2)

In [463]:
plot_presure_on_grid_LOOP(data, 112, WHITE, BLACK, BLACK, -0.4, 1.4, -0.3, 0.3, 'aggregation/actual_2')

In [464]:
folder_path = 'aggregation/actual_2'
mean_image = aggregate_images_mean(folder_path)
cv2.imwrite('aggregation/mean_image_from_other_airfoils.png', mean_image)

True

### Compare Metrics

In [593]:
# IoU
compare_airfoil_overlap('aggregation/result/mean_image_from_model_LOOP.png','aggregation/0027_actual_112_LOOP.png')

0.7088978396191871

In [466]:
compare_airfoil_overlap('aggregation/mean_image_from_other_airfoils.png','aggregation/0027_actual_112_LOOP.png')

0.9420743639921723

In [587]:
compare_airfoil_overlap('aggregation/extracted/1.png', 'aggregation/0027_actual_112_LOOP.png') # single

0.732211706975114

In [467]:
# SSIM
compute_ssim('aggregation/mean_image_from_model_LOOP.png','aggregation/0027_actual_112_LOOP.png')

0.8281972802542195

In [468]:
compute_ssim('aggregation/mean_image_from_other_airfoils.png','aggregation/0027_actual_112_LOOP.png')

0.9533368125054992

In [588]:
compute_ssim('aggregation/extracted/1.png', 'aggregation/0027_actual_112_LOOP.png')  # single

0.6873163876826218

In [469]:
# MSE
compute_mse('aggregation/mean_image_from_model_LOOP.png','aggregation/0027_actual_112_LOOP.png')

40.347735969387756

In [470]:
compute_mse('aggregation/mean_image_from_other_airfoils.png','aggregation/0027_actual_112_LOOP.png')

13.23126594387755

In [589]:
compute_mse('aggregation/extracted/1.png', 'aggregation/0027_actual_112_LOOP.png')  # single

80.77758290816327

In [471]:
# R
compute_r_squared('aggregation/mean_image_from_model_LOOP.png','aggregation/0027_actual_112_LOOP.png')

0.9922710859305456

In [472]:
compute_r_squared('aggregation/mean_image_from_other_airfoils.png','aggregation/0027_actual_112_LOOP.png')

0.9969190006743079

In [590]:
compute_r_squared('aggregation/extracted/1.png', 'aggregation/0027_actual_112_LOOP.png') # single

0.9862905969304264

In [473]:
# Overlay
overlay_difference('aggregation/mean_image_from_model_LOOP.png','aggregation/0027_actual_112_LOOP.png')

In [474]:
overlay_difference('aggregation/mean_image_from_other_airfoils.png','aggregation/0027_actual_112_LOOP.png')

In [591]:
overlay_difference('aggregation/extracted/1.png', 'aggregation/0027_actual_112_LOOP.png') # single

## Single Version

Use airfoil 0027 as main subject.

In [78]:
# subject
points_0027, triangles_0027, contour_0027, boundary_0027, pressure_coefs_0027 = read_all_data('0027.su2', '0027.csv')
# train
points_0028, triangles_0028, contour_0028, boundary_0028, pressure_coefs_0028 = read_all_data('0028.su2', '0028.csv')
points_0423, triangles_0423, contour_0423, boundary_0423, pressure_coefs_0423 = read_all_data('0423.su2', '0423.csv')
points_0427, triangles_0427, contour_0427, boundary_0427, pressure_coefs_0427 = read_all_data('0427.su2', '0427.csv')
points_0528, triangles_0528, contour_0528, boundary_0528, pressure_coefs_0528 = read_all_data('0528.su2', '0528.csv')
points_0729, triangles_0729, contour_0729, boundary_0729, pressure_coefs_0729 = read_all_data('0729.su2', '0729.csv')
points_2022, triangles_2022, contour_2022, boundary_2022, pressure_coefs_2022 = read_all_data('2022.su2', '2022.csv')
points_2230, triangles_2230, contour_2230, boundary_2230, pressure_coefs_2230 = read_all_data('2230.su2', '2230.csv')
points_2526, triangles_2526, contour_2526, boundary_2526, pressure_coefs_2526 = read_all_data('2526.su2', '2526.csv')
points_23122, triangles_23122, contour_23122, boundary_23122, pressure_coefs_23122 = read_all_data('23122.su2', '23122.csv')
points_24024, triangles_24024, contour_24024, boundary_24024, pressure_coefs_24024 = read_all_data('24024.su2', '24024.csv')                                                                                            

In [80]:
# 0027
plot_pressure_on_grid(points_0027, triangles_0027, pressure_coefs_0027, contour_0027, boundary_0027, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)
plot_black_and_white(points_0027, triangles_0027, pressure_coefs_0027, contour_0027, boundary_0027, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4) 
plot_pressure_on_grid(points_0027, triangles_0027, pressure_coefs_0027, contour_0027, boundary_0027, res=112, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)

# 0028
plot_pressure_on_grid(points_0028, triangles_0028, pressure_coefs_0028, contour_0028, boundary_0028, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)
plot_black_and_white(points_0028, triangles_0028, pressure_coefs_0028, contour_0028, boundary_0028, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4) 

# 0423
plot_pressure_on_grid(points_0423, triangles_0423, pressure_coefs_0423, contour_0423, boundary_0423, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)
plot_black_and_white(points_0423, triangles_0423, pressure_coefs_0423, contour_0423, boundary_0423, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4) 

# 0427
plot_pressure_on_grid(points_0427, triangles_0427, pressure_coefs_0427, contour_0427, boundary_0427, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)
plot_black_and_white(points_0427, triangles_0427, pressure_coefs_0427, contour_0427, boundary_0427, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4) 

# 0528
plot_pressure_on_grid(points_0528, triangles_0528, pressure_coefs_0528, contour_0528, boundary_0528, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)
plot_black_and_white(points_0528, triangles_0528, pressure_coefs_0528, contour_0528, boundary_0528, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4) 

# 0729
plot_pressure_on_grid(points_0729, triangles_0729, pressure_coefs_0729, contour_0729, boundary_0729, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)
plot_black_and_white(points_0729, triangles_0729, pressure_coefs_0729, contour_0729, boundary_0729, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4) 

# 2022
plot_pressure_on_grid(points_2022, triangles_2022, pressure_coefs_2022, contour_2022, boundary_2022, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)
plot_black_and_white(points_2022, triangles_2022, pressure_coefs_2022, contour_2022, boundary_2022, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4) 

# 2230
plot_pressure_on_grid(points_2230, triangles_2230, pressure_coefs_2230, contour_2230, boundary_2230, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)
plot_black_and_white(points_2230, triangles_2230, pressure_coefs_2230, contour_2230, boundary_2230, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4) 

# 2526
plot_pressure_on_grid(points_2526, triangles_2526, pressure_coefs_2526, contour_2526, boundary_2526, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)
plot_black_and_white(points_2526, triangles_2526, pressure_coefs_2526, contour_2526, boundary_2526, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4) 

# 23122
plot_pressure_on_grid(points_23122, triangles_23122, pressure_coefs_23122, contour_23122, boundary_23122, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)
plot_black_and_white(points_23122, triangles_23122, pressure_coefs_23122, contour_23122, boundary_23122, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4) 

# 24024
plot_pressure_on_grid(points_24024, triangles_24024, pressure_coefs_24024, contour_24024, boundary_24024, res=1000, start_color=WHITE, end_color=BLACK, airfoil_color=BLACK, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4)
plot_black_and_white(points_24024, triangles_24024, pressure_coefs_24024, contour_24024, boundary_24024, res=1000, inside_color=BLACK, outside_color=WHITE, x_min=-0.6, x_max=1.6, y_min=-0.4, y_max=0.4) 

In [90]:
# EXTRACT LAST IMAGE
img = extract_last_image("aggregation/0028_grid.png")
cv2.imwrite("extracted_0028.png", img)

img = extract_last_image("aggregation/0423_grid.png")
cv2.imwrite("extracted_0423.png", img)

img = extract_last_image("aggregation/0427_grid.png")
cv2.imwrite("extracted_0427.png", img)

img = extract_last_image("aggregation/0528_grid.png")
cv2.imwrite("extracted_0528.png", img)

img = extract_last_image("aggregation/0729_grid.png")
cv2.imwrite("extracted_0729.png", img)

img = extract_last_image("aggregation/2022_grid.png")
cv2.imwrite("extracted_2022.png", img)

img = extract_last_image("aggregation/2230_grid.png")
cv2.imwrite("extracted_2230.png", img)

img = extract_last_image("aggregation/2526_grid.png")
cv2.imwrite("extracted_2526.png", img)

img = extract_last_image("aggregation/23122_grid.png")
cv2.imwrite("extracted_23122.png", img)

img = extract_last_image("aggregation/24024_grid.png")
cv2.imwrite("extracted_24024.png", img)

In [94]:
def aggregate_images_mean(image_paths):
    image_sum = None
    for image_path in image_paths:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image_sum is None:
            image_sum = np.zeros_like(image, dtype=np.float64)
        image_sum += image
    mean_image = image_sum / len(image_paths)
    mean_image = np.clip(mean_image, 0, 255).astype(np.uint8)
    return mean_image

In [92]:
image_paths = ['extracted/extracted_0028.png','extracted/extracted_0423.png','extracted/extracted_0427.png','extracted/extracted_0528.png',
              'extracted/extracted_0729.png','extracted/extracted_2022.png','extracted/extracted_2230.png','extracted/extracted_23122.png',
              'extracted/extracted_24024.png','extracted/extracted_2526.png']
mean_image = aggregate_images_mean(image_paths)
cv2.imwrite('mean_image.png', mean_image)

### Aggregated

In [205]:
compare_airfoil_overlap('aggregation/mean_image.png','aggregation/0027_actual_112.png')

(1325, 1901, '69.70%')

In [206]:
compute_mse('aggregation/mean_image.png','aggregation/0027_actual_112.png')

43.35028698979592

In [207]:
compute_ssim('aggregation/mean_image.png','aggregation/0027_actual_112.png')

0.8410392406525449

In [208]:
compute_r_squared('aggregation/0027_actual_112.png', 'aggregation/mean_image.png')

'0.9888'

In [113]:
overlay_difference('aggregation/mean_image.png','aggregation/0027_actual_112.png')

### Single

In [204]:
# can change the choice of single image in the folder <extracted>
compare_airfoil_overlap('extracted/extracted_23122.png','aggregation/0027_actual_112.png')

(1127, 1777, '63.42%')

In [116]:
compute_mse('extracted/extracted_23122.png','aggregation/0027_actual_112.png')

68.95575573979592

In [117]:
compute_ssim('extracted/extracted_23122.png','aggregation/0027_actual_112.png')

0.794232118736409

In [118]:
compute_r_squared('aggregation/0027_actual_112.png', 'extracted/extracted_23122.png')

'0.9844'

In [119]:
overlay_difference('extracted/extracted_23122.png','aggregation/0027_actual_112.png')

# Use Similar Airfoils or Unsimilar Airfoils

Use 10 similar airfoils to predict 0027. Use 10 unsimilar airfoils to predict the same airfoil.

## Similar

In [475]:
FOLDER1 = 'alike/similar/original/'  # TO CHANGE
FOLDER2 = 'restart_csv/'  # TO CHANGE
data_similar = read_in_many_files(FOLDER1, FOLDER2)

In [476]:
# 右上
plot_presure_on_grid_LOOP(data_similar, 500, WHITE, BLACK, BLACK, -0.4, 1.4, -0.3, 0.3, 'alike/similar/actual')
# 左上  
plot_black_and_white_LOOP(data_similar, 500, BLACK, WHITE, -0.4, 1.4, -0.3, 0.3, 'alike/similar/bw')

In [483]:
input_folder = 'alike/similar/generated_grid'
output_folder = 'alike/similar/extracted/'
extract_last_image_LOOP(input_folder, output_folder)

## Unsimilar

In [480]:
FOLDER1 = 'alike/unsimilar/original/'  # TO CHANGE
FOLDER2 = 'restart_csv/'  # TO CHANGE
data_unsimilar = read_in_many_files(FOLDER1, FOLDER2)

In [481]:
# 右上
plot_presure_on_grid_LOOP(data_unsimilar, 500, WHITE, BLACK, BLACK, -0.4, 1.4, -0.3, 0.3, 'alike/unsimilar/actual')
# 左上  
plot_black_and_white_LOOP(data_unsimilar, 500, BLACK, WHITE, -0.4, 1.4, -0.3, 0.3, 'alike/unsimilar/bw')

In [484]:
input_folder = 'alike/unsimilar/generated_grid'
output_folder = 'alike/unsimilar/extracted/'
extract_last_image_LOOP(input_folder, output_folder)

## Compare metrics

In [505]:
# iou
predicted_folder = 'alike/similar/extracted'
actual_folder = 'alike/similar/LR_112'
iou_similar = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

predicted_folder = 'alike/unsimilar/extracted'
actual_folder = 'alike/unsimilar/LR_112'
iou_unsimilar = compare_airfoil_overlap_LOOP(predicted_folder, actual_folder)

print("IoU similar:", iou_similar)
print("IoU unsimilar:", iou_unsimilar)
print("\nIoU similar mean: {:.3f}".format(iou_similar.mean()))
print("IoU unsimilar mean: {:.3f}\n".format(iou_unsimilar.mean()))

t_statistic, p_value = stats.ttest_rel(iou_similar, iou_unsimilar)
print("t-statistic for similar and unsimilar: {:.4f}".format(t_statistic))
print("p-value for similar and unsimilar: {:.4f}".format(p_value))

IoU similar: [0.81549296 0.81538462 0.72863404 0.77106742 0.74399126 0.7621098
 0.80900457 0.77425044 0.76917808 0.73977695]
IoU unsimilar: [0.65846995 0.68376645 0.66869712 0.70479189 0.72892857 0.73364655
 0.70688456 0.68062317 0.74208633 0.73361227]

IoU similar mean: 0.773
IoU unsimilar mean: 0.704

t-statistic for similar and unsimilar: 4.2343
p-value for similar and unsimilar: 0.0022


In [507]:
# MSE
predicted_folder = 'alike/similar/extracted'
actual_folder = 'alike/similar/LR_112'
mse_similar = compute_mse_LOOP(predicted_folder, actual_folder)

predicted_folder = 'alike/unsimilar/extracted'
actual_folder = 'alike/unsimilar/LR_112'
mse_unsimilar = compute_mse_LOOP(predicted_folder, actual_folder)

print("MSE similar:", mse_similar)
print("MSE unsimilar:", mse_unsimilar)
print("\nMSE similar mean: {:.3f}".format(mse_similar.mean()))
print("MSE unsimilar mean: {:.3f}\n".format(mse_unsimilar.mean()))

t_statistic, p_value = stats.ttest_rel(mse_similar, mse_unsimilar)
print("t-statistic for similar and unsimilar: {:.4f}".format(t_statistic))
print("p-value for similar and unsimilar: {:.4f}".format(p_value))
print("SIGNIFICANT!")

MSE similar: [32.26618304 31.98294005 47.98612883 32.29328763 43.38185587 36.17753508
 34.2158801  32.21843112 34.63448661 33.8569037 ]
MSE unsimilar: [67.21635842 69.86981824 70.7869898  76.97608418 72.23580995 60.80524554
 64.90186543 74.94355867 85.21556122 77.49067283]

MSE similar mean: 35.901
MSE unsimilar mean: 72.044

t-statistic for similar and unsimilar: -12.2865
p-value for similar and unsimilar: 0.0000
SIGNIFICANT!


In [508]:
# R^2
predicted_folder = 'alike/similar/extracted'
actual_folder = 'alike/similar/LR_112'
r_similar = compute_r_squared_LOOP(predicted_folder, actual_folder)

predicted_folder = 'alike/unsimilar/extracted'
actual_folder = 'alike/unsimilar/LR_112'
r_unsimilar = compute_r_squared_LOOP(predicted_folder, actual_folder)

print("R^2 similar:", r_similar)
print("R^2 unsimilar:", r_unsimilar)
print("\nR^2 similar mean: {:.3f}".format(r_similar.mean()))
print("R^2 unsimilar mean: {:.3f}\n".format(r_unsimilar.mean()))

t_statistic, p_value = stats.ttest_rel(r_similar, r_unsimilar)
print("t-statistic for similar and unsimilar: {:.4f}".format(t_statistic))
print("p-value for similar and unsimilar: {:.4f}".format(p_value))
print("SIGNIFICANT!")

R^2 similar: [0.99071126 0.99151518 0.99075085 0.99074352 0.99124421 0.99181375
 0.99139851 0.99302711 0.99028966 0.99158272]
R^2 unsimilar: [0.98695398 0.9859423  0.98582768 0.98620747 0.98649519 0.98861768
 0.9889374  0.98731409 0.9866483  0.98659822]

R^2 similar mean: 0.991
R^2 unsimilar mean: 0.987

t-statistic for similar and unsimilar: 13.0401
p-value for similar and unsimilar: 0.0000
SIGNIFICANT!
